In [1]:
import logging
from azure.identity import DefaultAzureCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeResult
from src.interfaces.di_interface import DocIntInterface
import json
from datetime import datetime
import re
from azure.core.credentials import AzureKeyCredential

In [2]:
endpoint = "https://trv-di-pru-iagarantias-001.cognitiveservices.azure.com/"
credential = AzureKeyCredential("5zUCiMdRV1dAYM2Z21bD5Az47d2K3PEUCbVqtdTQ5UiWyzgMxt9iJQQJ99BBACYeBjFXJ3w3AAALACOGUFu1")
document_intelligence_client = DocumentIntelligenceClient(endpoint, credential)

In [25]:
with open("C:\\Users\\Xnd0x\\Documents\\ISA\\801046433_2_CONF800101_20250425152655589.pdf", "rb") as f:
    poller = document_intelligence_client.begin_analyze_document(model_id="prebuilt-read",body=f,pages="1-30")
# classification_result = poller.result()


In [26]:
classification_result = poller.result()

In [12]:
classification_result.documents[0]["docType"] == "Confianza"

True

In [3]:
with open("C:\\Users\\Xnd0x\\Documents\\ISA\\MINUTA_CONTRATO_CS-112_DATAKNOW_SAS.pdf", "rb") as f:
    poller = document_intelligence_client.begin_analyze_document(model_id="prebuilt-read", body=f,pages="1")


In [6]:
poller.status()


'running'

In [4]:
result: AnalyzeResult = poller.result()

In [5]:
result.content

'Docusign Envelope ID: A0E80020-8FB0-45B0-978B-B52BDC1B4E2C\nisa INTERCOLOMBIA\nCONTRATO MARCO CS-112 ISA: 4620004842 ITCO: 4620004841\nEl presente contrato se suscribe por ISA INTERCOLOMBIA S.A. E.S.P., en adelante LA EMPRESA actuando en nombre propio y a su vez actuando en nombre y representación de las siguientes empresas de INTERCONEXIÓN ELÉCTRICA S.A E.S.P (ISA), en virtud de los Acuerdos suscritos y a su vez de los mandatos con representación y los poderes vigentes entre estas sociedades y por consiguiente todos los efectos de este Contrato radican directamente en las empresas que lo suscriben.\nCuando en el presente contrato se use la expresión "LA EMPRESA", se entenderá que se refiere a ISA INTERCOLOMBIA S.A. E.S.P, cuando se estén regulando derechos y obligaciones propios o inherentes a la parte contratante como obligado principal y en su condición de mandante; tal es el caso de los asuntos relacionados con el cumplimiento de las obligaciones principales, la efectividad de las

In [9]:
from openai import AzureOpenAI

In [14]:
client = AzureOpenAI(
    api_key="9ZdC0OfiFYbjNGejuHcADRYqhpll2gUPd3OwradiqvsgNGSFkNNuJQQJ99BBACYeBjFXJ3w3AAABACOGmKlW",  
    api_version="2024-12-01-preview",
    azure_endpoint = "https://trv-oai-pru-iagarantias-001.openai.azure.com/"
    )

In [15]:
def CallId(content):
        logging.info(f"Content on AOI")
        response = client.chat.completions.create(
            model='o3-mini',
            messages=[
                {
                    "role": "system",
                    "content": """Necesito que analices la primera pagina de los documentos y logres identificar el tipo, existen solo 3: MinutaContrato,MinutaOrden y SAP, deberas retornar únicamente el tipo
                    Las diferencias son:
                    MinutaContrato: Unicamente viene el contrato en el archivo, no viene nada relacionado a orden
                    MinutaOrden: Viene el numero de orden siempre
                    SAP: Trae un codigo SAP
                    NOTA: A VECES EN MINUTA ORDEN VIENE EL TERMINO SAP, ESTO NO SIGNIFICA QUE SEA DEL TIPO SAP, SOLO SI TRAE EL CODIGO SAP
                    """,
                },
                {
                    "role": "user",
                    "content": "Extrae el objeto de este contenido fuente: " + content,
                }
            ]
        )
        logging.warning(f"Respuesta del OPENAI: {response.choices[0].message.content}")
        return  response.choices[0].message.content

In [16]:
CallId(result.content)

'MinutaContrato'

In [17]:
result = "MinutaContrato asdfasd "

In [20]:
# Lista de posibles opciones
opciones = ["MinutaContrato", "MinutaOrden", "SAP"]

# Buscar la primera coincidencia en el string
coincidencia = next((opcion for opcion in opciones if opcion in result), "MinutaContrato")


In [21]:
coincidencia

'MinutaContrato'

In [5]:
def formatear_fecha(fecha_str):
    if not fecha_str:
        return "01/01/2025"
    formatos = ["%d-%b-%Y", "%d-%m-%Y", "%d/%b/%Y", "%d/%m/%Y", "%d-%B-%Y", "%d/%B/%Y","%Y/%m/%d"]
    fecha = None
    for fmt in formatos:
        try:
            fecha = datetime.strptime(fecha_str, fmt)
            return fecha.strftime("%d/%m/%Y")
        except ValueError:
            pass
    SPANISH_MONTHS = {
        # abreviaturas
        "ENE": "01", "FEB": "02", "MAR": "03", "ABR": "04", "MAY": "05", "JUN": "06",
        "JUL": "07", "AGO": "08", "SEP": "09", "OCT": "10", "NOV": "11", "DIC": "12",
        # nombres completos
        "ENERO": "01", "FEBRERO": "02", "MARZO": "03", "ABRIL": "04", "MAYO": "05", "JUNIO": "06",
        "JULIO": "07", "AGOSTO": "08", "SEPTIEMBRE": "09", "OCTUBRE": "10", "NOVIEMBRE": "11", "DICIEMBRE": "12",
    }
    cad = fecha_str.strip().upper()
    def _repl(match):
        mes_txt = match.group(0)
        return SPANISH_MONTHS.get(mes_txt, mes_txt)
    cad_num = re.sub(r'\b[A-Z]{3,9}\b', _repl, cad)
    formatos2 = ["%d-%m-%Y", "%d/%m/%Y"]
    for fmt in formatos2:
        try:
            fecha = datetime.strptime(cad_num, fmt)
            return fecha.strftime("%d/%m/%Y")
        except ValueError:
            pass

    # 4) Si aun así no podemos, devolvemos el valor por defecto
    return "01/01/2025"
        
        
def limpiar_contrato(contrato_str):
    if not contrato_str:
        return 0
    match = re.search(r'\d{10,}', contrato_str)
    if match:
        return int(match.group())
    return 0
    


In [6]:
FIELD_MAP = {
        "Nacional": {
            "amparos":       "AMPAROS",
            "desde":         "VIGENCIA DESDE",
            "hasta":         "VIGENCIA HASTA",
            "valor_prima":   "SUMA ASEGURADA",
            "coberturas":"Coberturas",
            "contrato":"Contrato",
            "numeropoliza":"NumeroPoliza",
            "numeromodificacion":"NumeroModificacion",
            "aseguradora":"Aseguradora",
            "nombreproveedor":"NombreProveedor",
            "nitproveedor":"NitProveedor",
            "nombreasegurado":"NombreAsegurado",
            "nitasegurado":"NitAsegurado",
            "objetocaratula":"ObjetoCaratula"
        },
        "Bolivar": {
            "amparos":       "COBERTURA",
            "desde":         "DESDE",
            "hasta":         "HASTA",
            "valor_prima":   "VALOR ASEGURADO",
            "coberturas":"Coberturas",
                "contrato":"Contrato",
                "numeropoliza":"NumeroPoliza",
            "numeromodificacion":"NumeroModificacion",
            "aseguradora":"Aseguradora",
            "nombreproveedor":"NombreProveedor",
            "nitproveedor":"NitProveedor",
            "nombreasegurado":"NombreAsegurado",
            "nitasegurado":"NitAsegurado",
            "objetocaratula":"ObjetoCaratula"
        },
        "Cesce": {
            "amparos":       "Coberturas",
            "desde":         "DESDE",
            "hasta":         "HASTA",
            "valor_prima":   "Valor",
            "coberturas":"Coberturas",
            "contrato":"Contrato",
            "numeropoliza":"NumeroPoliza",
            "numeromodificacion":"NumeroModificacion",
            "aseguradora":"Aseguradora",
            "nombreproveedor":"NombreProveedor",
            "nitproveedor":"NitProveedor",
            "nombreasegurado":"NombreAsegurado",
            "nitasegurado":"NitAsegurado",
            "objetocaratula":"ObjetoCaratula"
        },
        "Chubb": {
            "amparos":       "Coberturas",
            "desde":         "Vig. Desde (YYYY/MM/DD)",
            "hasta":         "Vig. Hasta (YYYY/MM/DD)",
            "valor_prima":   "VLR. Asegurado",
            "coberturas":"Coberturas",
            "contrato":"Contrato",
            "numeropoliza":"NumeroPoliza",
            "numeromodificacion":"NumeroModificacion",
            "aseguradora":"Aseguradora",
            "nombreproveedor":"NombreProveedor",
            "nitproveedor":"NitProveedor",
            "nombreasegurado":"NombreAsegurado",
            "nitasegurado":"NitAsegurado",
            "objetocaratula":"ObjetoCaratula"
        },
        
            "Confianza": {
            "amparos":       "Cobertura",
            "desde":         "Desde",
            "hasta":         "Hasta",
            "valor_prima":   "Valor",
            "coberturas":"Coberturas",
            "contrato":"Contrato",
            "numeropoliza":"NumeroPoliza",
            "numeromodificacion":"NumeroModificacion",
            "aseguradora":"Aseguradora",
            "nombreproveedor":"NombreProveedor",
            "nitproveedor":"NitProveedor",
            "nombreasegurado":"NombreAsegurado",
            "nitasegurado":"NitAsegurado",
            "objetocaratula":"ObjetoCaratula"
        },
        "Solidaria": {
            "amparos":       "DESCRIPCION AMPAROS CONTRATO",
            "desde":         "VIGENCIA DESDE",
            "hasta":         "VIGENCIA HASTA",
            "valor_prima":   "SUMA ASEGURADA",
            "coberturas":"Coberturas",
            "contrato":"Contrato",
            "numeropoliza":"NumeroPoliza",
            "numeromodificacion":"NumeroModificacion",
            "aseguradora":"Aseguradora",
            "nombreproveedor":"NombreProveedor",
            "nitproveedor":"NitProveedor",
            "nombreasegurado":"NombreAsegurado",
            "nitasegurado":"NitAsegurado",
            "objetocaratula":"ObjetoCaratula"
        },
        "Sura2": {
            "amparos":       "COBERTURA",
            "desde":         "FECHA INICIAL",
            "hasta":         "FECHA VENCIMIENTO",
            "valor_prima":   "VALOR ASEGURADO",
            "coberturas":"Coberturas",
            "contrato":"Contrato",
            "numeropoliza":"NumeroPoliza",
            "numeromodificacion":"NumeroModificacion",
            "aseguradora":"Aseguradora",
            "nombreproveedor":"NombreProveedor",
            "nitproveedor":"NitProveedor",
            "nombreasegurado":"NombreAsegurado",
            "nitasegurado":"NitAsegurado",
            "objetocaratula":"ObjetoCaratula"
        },
        "Sura": {
            "amparos":       "Cobertura",
            "desde":         "FechaInicial",
            "hasta":         "FechaFinal",
            "valor_prima":   "Valor",
            "coberturas":"Coberturas",
                "contrato":"Contrato/Orden",
            "numeropoliza":"NumeroPoliza",
            "numeromodificacion":"NumeroModificacion",
            "aseguradora":"Aseguradora",
            "nombreproveedor":"NombreProveedor",
            "nitproveedor":"NitProveedor",
            "nombreasegurado":"NombreAsegurado",
            "nitasegurado":"NitAsegurado",
            "objetocaratula":"ObjetoCaratula"
        },
    }
DEFAULT_MAPPING = FIELD_MAP["Sura"]
    

In [7]:
output = {}

In [10]:
for doc in result.documents:
    if doc.fields is None:
        continue
    doctype = doc["docType"]
    # Determino el mapeo a usar
    mapping = FIELD_MAP.get(doc["docType"], DEFAULT_MAPPING)

    # Primero, copio todos los campos que NO sean Coberturas
    for name, fv in doc.fields.items():
        if name != "Coberturas":
            output[name] = fv.content or ""

    # Ahora, proceso el arreglo de Coberturas
    cob_array = doc.fields[mapping["coberturas"]]["valueArray"]  


    cober_list = [
        item["valueObject"][mapping["amparos"]].get("valueString","")
        for item in cob_array
        if mapping["amparos"] in item["valueObject"]
    ]
    desde_list = [
        item["valueObject"][mapping["desde"]].get("valueString","")
        for item in cob_array
        if mapping["desde"] in item["valueObject"]
    ]
    hasta_list = [
        item["valueObject"][mapping["hasta"]].get("valueString","")
        for item in cob_array
        if mapping["hasta"] in item["valueObject"]
    ]
    valor_list = [
        item["valueObject"][mapping["valor_prima"]].get("valueString","")
        for item in cob_array
        if mapping["valor_prima"] in item["valueObject"]
    ]
    unique_rows = []
       # 2) Empaquetamos en tuplas fila a fila
    rows = list(zip(cober_list, desde_list, hasta_list, valor_list))

    # 3) Filtramos duplicados preservando el primer encuentro
    seen = set()
    unique_rows = []
    for row in rows:
        if row not in seen:
            seen.add(row)
            unique_rows.append(row)
# 4) Desempaquetamos de nuevo en listas (o dejamos vacías si no quedó nada)
    if unique_rows:
        cober_u, desde_u, hasta_u, valor_u = zip(*unique_rows)
        output["coberturas"]      = list(cober_u)
        output["fechas_iniciales"]= list(desde_u)
        output["fechas_finales"]  = list(hasta_u)
        output["valores_asegurados"]= list(valor_u)
    else:
        output["coberturas"]       = []
        output["fechas_iniciales"] = []
        output["fechas_finales"]   = []
        output["valores_asegurados"]= []


In [20]:
output

{'NumeroPoliza': '801046433',
 'NombreProveedor': 'HMV INGENIEROS LIMITADA',
 'NitAsegurado': '900667590',
 'Moneda': 'PESOS',
 'Contrato': '4600004356',
 'Aseguradora': 'CONFIANZA',
 'NitProveedor': '860000656',
 'NombreAsegurado': 'ISA INTERCOLOMBIA S.A. E.S.P.',
 'ObjetoCaratula': 'AMPARAR EL PAGO DE LOS PERJUICIOS DERIVADOS DEL INCUMPLIMIENTO DE LAS OBLIGACIONES CONTENIDAS EN ELCONTRATO ABIERTO DE COMPRAVENTA, CONSTRUCCIÓN Y PRESTACIÓN\nDE SERVICIOS DE SUBESTACIONES No. 4600004356 ORDEN DE ENTREGA 4510022044, CUYO OBJETO ES LA EJECUCIÓN DE DISEÑOS, SUMINISTRO DE BIENES, OBRA CIVIL, MONTAJE, PRUEBAS Y PUESTA EN SERVICIO ASOCIADOS A LA NUEVA SUBESTACIÓN CDS2 230 KV.',
 'coberturas': ['Cumplimiento',
  'Pago salarios, prestaciones sociales e indemnizaciones laborales',
  'Estabilidad de la obra',
  'Provision de repuestos y accesorios'],
 'fechas_iniciales': ['11-04-2025', '11-04-2025', '22-09-2026', '22-09-2026'],
 'fechas_finales': ['22-10-2026', '22-09-2029', '22-09-2031', '22-09-2

In [21]:
  # Limpieza de posibles valores None
for k, v in output.items():
    if isinstance(v, list):
        output[k] = [elem or "" for elem in v]
    else:
        output[k] = v or ""

In [22]:
monedas = []
nuevos_valores_asegurados = []

In [23]:
for val in output.get('valores_asegurados',[]):
    moneda = output.get(f'Moneda', "")
    
    if val and "$" in val:
        partes = val.split("$", 1)
        moneda = partes[0].strip() if partes[0].strip() != "" else "COP"
        valor_numerico = partes[1].strip()
        monedas.append(moneda)
        nuevos_valores_asegurados.append(valor_numerico)
    else:
        if val.strip() != "":
            nuevos_valores_asegurados.append(val.strip())
            logging.warning(f"Moneda strip {moneda.strip()}")
            if moneda and moneda.strip() != "":
                if(moneda.strip() == "COL$"):
                    monedas.append("COP")
                    continue
                if(moneda.strip() == "$US"):
                    monedas.append("USD")
                    continue
                if(moneda.strip() == "PESOS"):
                    monedas.append("COP")
                    continue
                if(moneda.strip() == "DOLARES"):
                    monedas.append("USD")
                    continue
                if(moneda.strip() == "$"):
                    monedas.append("COP")
                    continue
                if(moneda.strip() == "US$"):
                    monedas.append("USD")
                    continue
                if(moneda.strip() == "US $"):
                    monedas.append("USD")
                    continue
                if(moneda.strip() == "($-Pesos"):
                    monedas.append("COP")
                    continue
                if(moneda.strip() == "($USA-Dolares"):
                    monedas.append("USD")
                    continue
                if(moneda.strip() == "EUR$"):
                    monedas.append("EUR")
                    continue
                else:
                    monedas.append("COP")
                    continue
            else:
                monedas.append("COP")
        else:
            nuevos_valores_asegurados.append("")
            monedas.append("")

# Se eliminan los campos específicos de coberturas del output general

In [24]:
for i in range(1, 10):
    sufijo = "" if i == 1 else str(i)
    for campo in [f'Cobertura{sufijo}', f'FechaInicialCobertura{sufijo}', f'FechaFinalCobertura{sufijo}', f'ValorAseguradoMoneda{sufijo}']:
        if campo in output:
            output.pop(campo)


In [25]:
resultados = []
m = FIELD_MAP.get(doctype)
if not m:
    raise ValueError(f"No existe configuración de campos para el doctype {doctype!r}")
logging.warning(f"dctype {doctype}")
logging.warning(f"map {m}")

# Obtener cantidad de coberturas encontradas (se asume que todas las listas tienen la misma longitud)
cantidad = len(output.get("coberturas",[]))
logging.warning(f"cantidad {cantidad}")
logging.warning(f"cantidad { output.get('coberturas',[]) }")
logging.warning(f"fechainicial {output.get('fechas_iniciales',[])}")
logging.warning(f"fechafinal {output.get('fechas_iniciales',[])}")


logging.warning(f"Valores : {nuevos_valores_asegurados}")
logging.warning(f"Monedas : {monedas}")
for idx in range(cantidad):
    obj_cobertura = {}
    obj_cobertura["NumeroPóliza"] = output.get(m["numeropoliza"], "")
    obj_cobertura["NumeroModificación"] = output.get(m["numeromodificacion"], "")
    obj_cobertura["Aseguradora"] = output.get(m["aseguradora"], "")
    obj_cobertura["NombreProveedor"] = output.get(m["nombreproveedor"], "")
    obj_cobertura["NitProveedor"] = output.get(m["nitproveedor"], "")
    obj_cobertura["NombreAsegurado"] = output.get(m["nombreasegurado"], "")
    obj_cobertura["NitAsegurado"] = output.get(m["nitasegurado"], "")
    obj_cobertura["ObjetoCaratula"] = output.get(m["objetocaratula"], "")
    logging.warning("SETTING COBERTURAS")

    obj_cobertura["Cobertura"] = output.get("coberturas",[])[idx]
    logging.warning("SETTING fechas")
    obj_cobertura["FechaInicialCobertura"] = formatear_fecha(output.get('fechas_iniciales',[])[idx])
    obj_cobertura["FechaFinalCobertura"] = formatear_fecha(output.get('fechas_finales',[])[idx])

    
    obj_cobertura["ValorAsegurado"] = nuevos_valores_asegurados[idx]
    obj_cobertura["Moneda"] = monedas[idx]

    obj_cobertura["ContratoOrden"] = limpiar_contrato(output.get(m["contrato"], ""))
    resultados.append(obj_cobertura)

In [26]:

finaljson = json.dumps(resultados)

In [27]:
finaljson

'[{"NumeroP\\u00f3liza": "801046433", "NumeroModificaci\\u00f3n": "", "Aseguradora": "CONFIANZA", "NombreProveedor": "HMV INGENIEROS LIMITADA", "NitProveedor": "860000656", "NombreAsegurado": "ISA INTERCOLOMBIA S.A. E.S.P.", "NitAsegurado": "900667590", "ObjetoCaratula": "AMPARAR EL PAGO DE LOS PERJUICIOS DERIVADOS DEL INCUMPLIMIENTO DE LAS OBLIGACIONES CONTENIDAS EN ELCONTRATO ABIERTO DE COMPRAVENTA, CONSTRUCCI\\u00d3N Y PRESTACI\\u00d3N\\nDE SERVICIOS DE SUBESTACIONES No. 4600004356 ORDEN DE ENTREGA 4510022044, CUYO OBJETO ES LA EJECUCI\\u00d3N DE DISE\\u00d1OS, SUMINISTRO DE BIENES, OBRA CIVIL, MONTAJE, PRUEBAS Y PUESTA EN SERVICIO ASOCIADOS A LA NUEVA SUBESTACI\\u00d3N CDS2 230 KV.", "Cobertura": "Cumplimiento", "FechaInicialCobertura": "11/04/2025", "FechaFinalCobertura": "22/10/2026", "ValorAsegurado": "3,837,960,996.40", "Moneda": "COP", "ContratoOrden": 4600004356}, {"NumeroP\\u00f3liza": "801046433", "NumeroModificaci\\u00f3n": "", "Aseguradora": "CONFIANZA", "NombreProveedor"

In [18]:
from openai import AzureOpenAI

In [19]:
client = AzureOpenAI(
    api_key="9ZdC0OfiFYbjNGejuHcADRYqhpll2gUPd3OwradiqvsgNGSFkNNuJQQJ99BBACYeBjFXJ3w3AAABACOGmKlW",  
    api_version="2024-10-21",
    azure_endpoint = "https://trv-oai-pru-iagarantias-001.openai.azure.com/"
    )

In [61]:
content = """Docusign Envelope ID: A0E80020-8FB0-45B0-978B-B52BDC1B4E2C
isa INTERCOLOMBIA
CONTRATO MARCO CS-112 ISA: 4620004842 ITCO: 4620004841
El presente contrato se suscribe por ISA INTERCOLOMBIA S.A. E.S.P., en adelante LA EMPRESA actuando en nombre propio y a su vez actuando en nombre y representación de las siguientes empresas de INTERCONEXIÓN ELÉCTRICA S.A E.S.P (ISA), en virtud de los Acuerdos suscritos y a su vez de los mandatos con representación y los poderes vigentes entre estas sociedades y por consiguiente todos los efectos de este Contrato radican directamente en las empresas que lo suscriben.
Cuando en el presente contrato se use la expresión "LA EMPRESA", se entenderá que se refiere a ISA INTERCOLOMBIA S.A. E.S.P, cuando se estén regulando derechos y obligaciones propios o inherentes a la parte contratante como obligado principal y en su condición de mandante; tal es el caso de los asuntos relacionados con el cumplimiento de las obligaciones principales, la efectividad de las garantías, las retenciones y descuentos pactados en el contrato y la aplicación de la condición resolutoria, entre otras.
Así mismo, se entenderá que la expresión "LA EMPRESA" se refiere a ISA INTERCOLOMBIA S.A. E.S.P, cuando regule la ejecución de actividades propias de la condición de mandatario con representación de la parte contratante, tales como la gestión de documentos, la supervisión del contrato, la facturación y los pagos, entre otras.
De otro lado, se entenderá que la expresión "LA EMPRESA" se refiere indistintamente a mandante y mandatario, cuando regula derechos y obligaciones que atañen conjuntamente a ambas personas jurídicas como consecuencia del mandato, como en el caso de la protección de datos personales, la propiedad intelectual, la confidencialidad, la indemnidad, entre otros.
Entre los suscritos DIANA ISABEL PIZARRO mayor de edad, domiciliada en Medellín, identificada con cédula de ciudadanía 32.350.854 quien en su calidad de Directora de Servicios de aprovisionamiento obra en nombre y representación de ISA INTERCOLOMBIA S.A. E.S.P., Empresa de Servicios Públicos Mixta, constituida mediante la Escritura Pública 1.584 del 9 de Octubre del 2013 de la Notaría Única de Sabaneta, sociedad anónima de carácter comercial, domiciliada en Medellín - Colombia, sometida al régimen jurídico establecido por las Leyes 142 y 143 de 1994 y a las normas de derecho privado, identificada con Nit: 900667590-1, que para los efectos del presente documento se denominará LA EMPRESA, y por la otra, CARLOS ANDRES MURILLO GALLEGO, mayor de edad, domiciliado en Medellín, identificado con Cédula de ciudadanía número 75.105.636, quién en su calidad de Representante Legal, obra en nombre y representación de DATA KNOW S.A.S, sociedad comercial constituida por acciones simplificada mediante documento privado el 7 de septiembre de 2018, del apoderado, registrado en la Cámara de Comercio de Medellín para Antioquia el10 de septiembre de 2018, en el libro IX, bajo el número 24464, la sociedad identificada con Nit: 901212206-9, que para efectos de este documento se denominará el CONTRATISTA, y como tal declara, bajo la gravedad del juramento, que no está incurso en ninguna de las inhabilidades e incompatibilidades
ISA INTERCOLOMBIA S.A. E.S.P. NIT: 900.667.590-1
Calle 12 Sur 18 - 168 Bloque 1 Pisos 2 y 3 Medellín, Colombia Tel: + 57 4 3252400
Línea única de atención: 018000 942001 Línea ética: opción 3
lineaetica@intercolombia.com
www.intercolombia.com
Docusign Envelope ID: A0E80020-8FB0-45B0-978B-B52BDC1B4E2C
isa INTERCOLOMBIA
previstas legalmente para la contratación estatal y que en su contra no se ha dictado fallo con responsabilidad fiscal, según lo indicado en el Artículo 60 de la Ley 610 de 2000; se ha celebrado el Contrato servicios, contenido en las siguientes cláusulas:
1. DEFINICIONES.
En este Contrato los términos tendrán los significados que se indican a continuación. Toda referencia a "Numeral", "Punto", "Inciso", "Formulario" y "Anexo", se deberá entender efectuada a numerales, puntos, incisos, formularios y anexos de este contrato o de los documentos de la etapa precontractual, según se indique; salvo disposición expresa en otro sentido. Las expresiones en singular comprenden, en su caso, al plural y viceversa. Los términos que figuren en mayúsculas en el presente contrato y que no se encuentren expresamente definidos en este, corresponden a Normas Aplicables o a términos usualmente utilizados en mayúscula:
"Acuerdos de Niveles de Servicios -ANS-": Son protocolos pactados por las partes que permiten establecer modelos de cumplimiento en la ejecución del objeto del Contrato, basados en un compromiso que puede ser medido y demostrado objetivamente.
"Acuerdo de Transacción": Acto jurídico bilateral por medio del cual las partes solucionan una controversia contractual, haciéndose concesiones recíprocas con la intención de precaver un eventual litigio.
"Autoridad": Es cualquier persona o entidad investida de poder público, facultada conforme a las Leyes Aplicables para implementar, emitir o interpretar normas o decisiones, generales o particulares, con efectos obligatorios para quienes se encuentren sometidos a sus alcances. Cualquier mención a una autoridad específica deberá entenderse efectuada a ésta o a quien la suceda o a quien ésta designe para realizar los actos a que se refiere este contrato o las Leyes Aplicables.
"Contrato de Construcción": Aquel que tiene por objeto la fabricación, adecuación o mejora de una edificación o infraestructura, incluyendo las obras accesorias como las redes eléctricas y de datos, alcantarillado, acueducto, siempre que una vez realizadas, no pueden ser retiradas del suelo o del inmueble al que adhieren, porque de hacerlo se afectaría su funcionamiento.
"Contrato de Compraventa": Es el contrato por el cual una parte denominada vendedor se obliga a entregar a la otra parte, una cosa determinada, a cambio de que el comprador le pague un precio cierto en dinero.       
"Contrato de objeto compuesto": Modalidad de contrato en el que se ejecutan dos o más actividades.
"Contrato de Prestación de Servicios": Es el contrato mediante el cual una persona natural o jurídica, normalmente especialista en algún área, se obliga con otra a ejecutar actividades (objeto inmaterial), a cambio de un precio que se materializa por el cumplimiento de metas, horas, objetivos, proyectos, etc.
ISA INTERCOLOMBIA S.A. E.S.P. NIT: 900.667.590-1
Calle 12 Sur 18 - 168 Bloque 1 Pisos 2 y 3 Medellín, Colombia Tel: + 57 4 3252400
Línea única de atención: 018000 942001 Línea ética: opción 3
lineaetica@intercolombia.com
www.intercolombia.com
Docusign Envelope ID: A0E80020-8FB0-45B0-978B-B52BDC1B4E2C
isa INTERCOLOMBIA
"Contrato de suministro": Contrato por medio del cual una persona natural o jurídica se obliga, a cambio de un pago o contraprestación por parte de la otra a cumplir a favor de ésta prestaciones periódicas o continuadas de bienes o servicios.
"Costos Locales": Corresponden a aquellas erogaciones accesorias, producto de la importación, sin incluir los impuestos aduaneros (Arancel e IGV). Entre otros, están compuestos por el transporte entre el puerto o lugar de arribo al territorio nacional y el lugar de destino de la mercancía o del bien, bodegaje, contratación de seguridad que escolte el bien, etc.
"Documento de Solicitud de Ofertas": Documento que da inicio a la etapa precontractual, de carácter exhaustivo y vinculante porque se entiende formar parte integral del contrato, que recoge las exigencias de índole técnica, comercial y legal que han de regir la formación del acuerdo de voluntades, el perfeccionamiento del negocio jurídico y la posterior ejecución del contrato.
"Especificaciones técnicas": Son los documentos por medio de los cuales ISA o LA EMPRESA, según corresponda, le definen al(os) CONTRATISTA(S) las normas, exigencias y procedimientos a ser empleados en la construcción de obras, la fabricación y el suministro de bienes y la prestación de servicios.
"INCOTERMS": (International Commercial Terms, 'términos internacionales de comercio') Son términos de tres letras cada uno que reflejan las normas, de aceptación voluntaria por las dos partes -compradora y vendedora-, acerca de las condiciones de entrega de mercancías. Se usan para aclarar los costos de las transacciones comerciales internacionales, delimitando las responsabilidades entre el comprador y el vendedor, y reflejan la práctica actual en el transporte internacional de mercancías. Solo son utilizados para la compraventa de mercaderías internacionales.
"Importación": Ingresar al país bienes o mercancías de origen extranjero para uso o consumo de la empresa o para comercializarlos a terceros.
"Importador": Quién adelanta las gestiones para el ingreso de la mercancía al territorio nacional. No necesariamente es el propietario de la mercancía, es quien presta el servicio logístico para la importación.      
"Ley Aplicable": Son individualmente y en conjunto todas las normas jurídicas que conforman el derecho interno del país en el cual se ejecutará el contrato, así como sus normas complementarias, supletorias y modificatorias, que tengan relación con el contrato.
"Matriz de Riesgos": Es el instrumento por medio del cual se tipifican, estiman, valoran y asignan los riesgos previsibles en la contratación y se definen las correspondientes medidas de administración.
"Obligación principal": Es la conformada por el objeto, alcance y plazo del contrato.
"Obligaciones accesorias": Se refiere a las demás obligaciones pactadas por las partes, además del objeto, alcance y plazo del contrato.
ISA INTERCOLOMBIA S.A. E.S.P. NIT: 900.667.590-1
Calle 12 Sur 18 - 168 Bloque 1 Pisos 2 y 3 Medellín, Colombia Tel: + 57 4 3252400
Línea única de atención: 018000 942001 Línea ética: opción 3
lineaetica@intercolombia.com
www.intercolombia.com
Docusign Envelope ID: A0E80020-8FB0-45B0-978B-B52BDC1B4E2C
isa INTERCOLOMBIA
"Orden de Ejecución u Orden de Entrega": Documento mediante el cual se ejecutan los pedidos abiertos celebrados por LA EMPRESA, según corresponda, y contiene las condiciones específicas de suministro o entrega del bien o ejecución del servicio al que hace referencia.
"Otros acuerdos contractuales": Hacen referencia a las manifestaciones formales de las partes en las que constan sus acuerdos en relación con la manera precisa de ejecutar las prestaciones objeto del contrato, tales como las actas de avance de obra, las actas de reuniones técnicas de revisión de ingeniería o de diseño, lo consignado en las bitácoras de ejecución de obra, entre otros, siendo esta lista meramente enunciativa a título de ejemplos.
"Parte o Partes": Es, según sea el caso, individual o conjuntamente, ISA o LA EMPRESA, según corresponda, de un lado, y el CONTRATISTA del otro.
"Días": Son los días que no sean sábado, domingos o feriados en Colombia, salvo indicación expresa en sentido contrario. Para los efectos de este Contrato, las referencias horarias se hacen a las horas de Colombia. Aplica lo establecido en los artículos 59 y siguientes del Código de Régimen Político Municipal y lo preceptuado por el artículo 829 del Código de Comercio.
"Riesgos intrínsecos al negocio, ordinarios o del aleas normal de la actividad del CONTRATISTA": Es la posibilidad de materialización de un evento previsible, de común ocurrencia dentro de la actividad objeto del contrato, que carece de entidad suficiente para hacer excesivamente oneroso el cumplimiento de la obligación por parte del CONTRATISTA y que es valorada a priori por éste, al ser incluida como uno de los componentes que conforman los valores de su oferta y por ende, los precios del contrato
2. OBJETO.
El objeto de la presente solicitud es la prestación de servicios de soporte, gestión, administración, monitoreo, mantenimiento y desarrollo de RPAs y analítica avanzada para ISA y sus empresas.
3. ALCANCE.
Grupo 1: Analítica Predictiva, Prescriptiva e Inteligencia Artificial
Desarrollar y realizar mantenimiento y soporte de soluciones de analítica predictiva, prescriptiva e inteligencia artificial. Para implementar innovaciones que optimicen procesos y mejoren la toma de decisiones estratégicas en sectores como energía, vías, telecomunicaciones, finanzas, aprovisionamiento, talento organizacional, jurídica, entre otros.
El alcance para la prestación de los servicios y las especificaciones técnicas de los mismos, se detallan en los siguientes anexos:
- Anexo0_Glosario Tecnico
- Anexo1_Marco general y especificaciones técnicas
- Anexo2_Sedes y horarios
ISA INTERCOLOMBIA S.A. E.S.P. NIT: 900.667.590-1
Calle 12 Sur 18 - 168 Bloque 1 Pisos 2 y 3 Medellín, Colombia Tel: + 57 4 3252400
Línea única de atención: 018000 942001 Línea ética: opción 3 lineaetica@intercolombia.com
www.intercolombia.com
Docusign Envelope ID: A0E80020-8FB0-45B0-978B-B52BDC1B4E2C
isa INTERCOLOMBIA
- Anexo4_Informes
- Anexo5_TransicionServicio
- Anexo8_ANS ICD
- Anexo9_ModeloGobiernoDevOps
Anexo 10_ModeloGobiernoDataLake -
- Anexo 11_DocumentoEjecucionOE
CONDICIONES PARA SOLICITAR, EVALUAR Y ADJUDICAR ÓRDENES DE ENTREGA
1. Para llevar a cabo la prestación de los servicios objeto del contrato, se suscribirán contratos marcos para las empresas ISA e ISA-INTERCOLOMBIA, los cuales serán ejecutados mediante órdenes de servicios.
2. Durante la ejecución del contrato abierto para solicitar, evaluar y emitir las órdenes de entrega, LA EMPRESA según el medio que defina (Ariba, físico o correo electrónico), requerirá a los Proveedores seleccionados para que coticen los servicios objeto del contrato, para lo que establecerá la descripción, especificaciones y características técnicas que deben cumplir los servicios, cantidades, plazos, entre otros, según aplique.
3. Para la presentación de la información requerida para evaluar los aspectos anteriores, se enviará a los Proveedores toda la información necesaria para que puedan presentar su cotización.
4. Las órdenes de entrega podrán ser emitidas a uno o varios Proponentes, siempre y cuando éstos cumplan con las condiciones establecidas en la solicitud de cotización.
5. Los Proveedores deberán estar dispuestos a presentar las cotizaciones usando los formatos y medios que defina LA EMPRESA en la ejecución de los contratos.
6. La suscripción de contratos abiertos no constituye para LAS EMPRESAS un pacto de exclusividad ni la obligación de adjudicar ordenes de entrega a los contratistas.
7. LAS EMPRESAS se asegurarán de que los valores de los servicios a adquirir mediante las órdenes de entrega estén ajustados a las condiciones objetivas del mercado y en el evento de no estarlo, o cuando LAS EMPRESAS lo consideren pertinente, se reservan el derecho a adelantar otros procesos de contratación para la adquisición de esos servicios en las condiciones más favorables; por lo que podrá no invitar a cotizar o declarar inconveniente la solicitud de cotizaciones cuando a su criterio observe que, con los contratistas que tengan contrato abierto, no conseguirá las mejores condiciones objetivas del mercado; sin que por ello se generen el pago de indemnizaciones o perjuicios, de manera
ISA INTERCOLOMBIA S.A. E.S.P. NIT: 900.667.590-1
Calle 12 Sur 18 - 168 Bloque 1 Pisos 2 y 3 Medellín, Colombia Tel: + 57 4 3252400
Línea única de atención: 018000 942001 Línea ética: opción 3
lineaetica@intercolombia.com
www.intercolombia.com
Docusign Envelope ID: A0E80020-8FB0-45B0-978B-B52BDC1B4E2C
isa INTERCOLOMBIA
que LAS EMPRESAS en ningún caso garantizan o se comprometen ante los contratistas con contrato abierto, en la obtención de ingresos o utilidades de ninguna índole.
4. PLAZO.
La vigencia del contrato será de cinco (5) años contados a partir de la fecha de la suscripción del presente documento. Por su parte los plazos serán determinados en cada orden de servicios. Los plazos previstos en las órdenes de servicios no podrán superar la vigencia del contrato.
La modificación de la vigencia del contrato y del plazo de las órdenes se realizará por acuerdo entre las partes mediante Cláusula Adicional.
Parágrafo - Las condiciones de empalme están definidas en el Anexo 5 - Transición del servicio. Esta etapa se ejecutará a partir de la fecha señalada en la orden de iniciación.
5. VALOR.
El valor del Contrato es indeterminado, pero determinable a su vencimiento, con base en la sumatoria de todos los valores liquidados y facturados por el Contratista, de conformidad con la sumatoria del valor de cada orden de servicios y los dispuesto en el Numeral 5. Forma y Condiciones de Pago.
Este valor no incluye el impuesto sobre las ventas (IVA), el cual será liquidado según la norma tributaria vigente.
Los precios del Contrato serán reajustados a partir del 1 de enero de cada año, en un porcentaje equivalente al incremento en el Índice de Precios al Consumidor -IPC- certificado por el DANE para el año inmediatamente anterior.
6. FORMA Y CONDICIONES DE PAGO
Las facturas podrán constar en documentos físicos y/o medios electrónicos, según el caso, en los términos definidos por la normatividad vigente. EL CONTRATISTA le otorga validez probatoria a los registros electrónicos generados por el sistema transaccional designado por LA EMPRESA para tal fin, con base en las transacciones realizadas con la información suministrada por él, de conformidad con los términos que aquí se expresan y los establecidos en las normas aplicables a la materia.
EL CONTRATISTA deberá tener en cuenta que LA EMPRESA admitirá firma digital como elemento para garantizar autenticidad e integridad de la factura electrónica desde su expedición hasta su conservación, por lo que deberá contar con certificado digital vigente y
ISA INTERCOLOMBIA S.A. E.S.P. NIT: 900.667.590-1
Calle 12 Sur 18 - 168 Bloque 1 Pisos 2 y 3 Medellín, Colombia Tel: + 57 4 3252400
Línea única de atención: 018000 942001 Línea ética: opción 3 lineaetica@intercolombia.com
www.intercolombia.com
Docusign Envelope ID: A0E80020-8FB0-45B0-978B-B52BDC1B4E2C
isa INTERCOLOMBIA
no revocado al momento de la firma de conformidad con la normatividad aplicable. La firma digital que se incluya en la factura podrá pertenecer al obligado, a los sujetos autorizados en su empresa o al proveedor tecnológico expresamente autorizado.
Para el caso de contratos de mandato y contratos suscritos con CONTRATISTAS en cualquiera de las formas de colaboración empresarial, deberá aplicarse lo dispuesto por el artículo 11 del Decreto 2242 de 2015 y las normas que los sustituyan, adicionan o modifiquen.
Para los servicios fijos de soporte y mantenimiento El Contratista presentará factura mensual a nombre de ISA INTERCOLOMBIA S.A E.S.P o de INTERCONEXIÓN ELECTRICA S.A. E.S.P por los servicios prestados en el mes inmediatamente anterior indicados en la orden de entrega de servicios una vez sean recibidos a satisfacción de LA EMPRESA, de acuerdo al Formulario C.1A -Valores mensuales, luego de aplicar al valor a facturar el porcentaje indicado (si aplica) en el Anexo 01 - Gobierno y Marco General para la prestación del servicio.
Para los Proyectos y trabajos especiales El Contratista presentará facturas a nombre de ISA INTERCOLOMBIA S.A E.S.P o de INTERCONEXIÓN ELÉCTRICA S.A. E.S.P, según la empresa que emita la orden de servicios.
Todas las facturas deberán tramitarse acompañadas de los documentos requeridos para su trámite y de la Constancia de Cumplimiento a satisfacción de LA EMPRESA firmada por el DIRECTOR (A) TECNOLOGÍA DE INFORMACIÓN.   
Para el impuesto sobre las ventas, en la prestación de servicios, el porcentaje de IVA será el establecido en la norma tributaria vigente al momento de su causación.
La factura podrá presentarse de acuerdo a las siguientes opciones:
· En las oficinas de LA EMPRESA en la ciudad de Medellín.
· A través del correo electrónico dispuesto por LA EMPRESA para tal fin, para los obligados a facturar electrónicamente.
La factura deberá tramitarse acompañada de los documentos requeridos para su trámite y de la Constancia de Cumplimiento a satisfacción de LA EMPRESA firmada por la DIRECTOR (A) TECNOLOGÍA DE INFORMACIÓN.
Para el impuesto sobre las ventas, en la prestación de servicios, el porcentaje de IVA será el establecido en la norma tributaria vigente al momento de su causación.
ISA INTERCOLOMBIA S.A. E.S.P. NIT: 900.667.590-1
Calle 12 Sur 18 - 168 Bloque 1 Pisos 2 y 3 Medellín, Colombia Tel: + 57 4 3252400
Línea única de atención: 018000 942001 Línea ética: opción 3
lineaetica@intercolombia.com
www.intercolombia.com
Docusign Envelope ID: A0E80020-8FB0-45B0-978B-B52BDC1B4E2C
isa INTERCOLOMBIA
OBLIGACIONES PACTADAS EN PESOS COLOMBIANOS Y PAGADERAS EN PESOS COLOMBIANOS
Las facturas serán canceladas dentro de los treinta (30)) días calendario siguientes a la fecha de su radicación física o electrónica. Este pago lo realizará la Tesorería dentro de la semana del vencimiento de la factura, acorde con las Políticas de Pago de la Empresa.
Cuando LA EMPRESA no cancele el valor de la factura dentro de la semana del vencimiento estipulado en esta cláusula, reconocerá sobre el saldo y proporcional al tiempo de la mora, la tasa DTF a noventa (90) días efectiva anual, vigente a la semana de inicio del período moratorio acorde con las políticas de pago de LA EMPRESA.
Factura física - título valor
Si la factura física es título valor, el Contratista la presentará en original y dos (2) copias, acompañada de los documentos que se exigieren en esta cláusula. En el original y en las copias de la factura se colocará un sello de "RECIBIDO", que indicará fecha, hora y firma de quién recibió. El original y una copia de la factura, se devolverán al Contratista y la otra copia quedará en LA EMPRESA. El documento de aceptación de la factura como título valor, se enviará al Contratista dentro de los tres (3) días hábiles siguientes a la fecha de recepción de la factura (ver artículo 86 de la Ley 1676 de 2013), firmado por el DIRECTOR (A) TECNOLOGÍA DE INFORMACIÓN de LA EMPRESA.
Cuando el Contratista endose la factura título valor, el legítimo tenedor del título, para su pago, deberá presentar en las oficinas de LA EMPRESA en la ciudad de Medellín, como mínimo con tres (3) días calendario de anticipación a la fecha de pago, el original de la factura, acompañada de los siguientes documentos:
Persona natural: Fotocopia simple de su cédula de ciudadanía y una comunicación suscrita por el legítimo tenedor o por su representante legal si éste es persona jurídica, en la que se indique el nombre del banco, tipo de cuenta (ahorros, corriente u otras), número de la cuenta, NIT o cédula, teléfono y ciudad.
Persona jurídica: Original o fotocopia simple del Certificado de Existencia y Representación Legal, expedido por el organismo competente con una antelación no superior a treinta (30) días calendario, con respecto a la fecha de su presentación ante LA EMPRESA y una comunicación suscrita por el legítimo tenedor o por su representante legal si éste es persona jurídica, en la que se indique el nombre del banco, tipo de cuenta (ahorros, corriente u otras), número de la cuenta, NIT o cédula, teléfono y ciudad.
LA EMPRESA sólo recibirá la factura lunes a jueves, de 8:00 a.m. a 4:30 p.m
ISA INTERCOLOMBIA S.A. E.S.P. NIT: 900.667.590-1
Calle 12 Sur 18 - 168 Bloque 1 Pisos 2 y 3 Medellín, Colombia Tel: + 57 4 3252400
Línea única de atención: 018000 942001 Línea ética: opción 3
lineaetica@intercolombia.com
www.intercolombia.com
Docusign Envelope ID: A0E80020-8FB0-45B0-978B-B52BDC1B4E2C
isa INTERCOLOMBIA
El original de la factura título valor remitida por correo se radicará en LA EMPRESA y junto con el documento de aceptación correspondiente, sólo se entregará personalmente al emisor, a su Representante Legal o a su apoderado para el efecto.
Factura electrónica - título valor
Cuando la factura electrónica sea título valor, para su comercialización las partes tendrán en cuenta lo dispuesto por el Decreto 2242 de 2015 expedido por el Ministerio de Hacienda y Crédito Público, el Manual de funcionamiento del administrador de facturas electrónicas definido en la Resolución 2215 de 2017 expedida por el Ministerio de Comercio, Industria y Turismo, el Manual de funcionamiento de los sistemas de negociación electrónica contenido en la Resolución 294 de 2018 expedida por el Ministerio de Comercio, Industria y Turismo y el correspondiente manual del usuario que expida el del administrador del sistema de facturación electrónica y demás normas que las modifiquen, adicionen o sustituyan.
Cuando se requiera tramitar la factura como título valor el Contratista deberá enviar al buzón facturasproveedores@intercolombia.com adjunto a la factura comunicación escrita informando la intención de negociación e informar en el texto del correo.
El documento de aceptación de la factura como título valor, se enviará al Contratista dentro de los tres (3) días hábiles siguientes a la fecha de recepción de la factura (ver artículo 86 de la Ley 1676 de 2013), firmado por el DIRECTOR (A) TECNOLOGÍA DE INFORMACIÓN de LA EMPRESA.
Cuando el Contratista endose la factura título valor, el legítimo tenedor del título, para su pago, deberá presentar en las oficinas de LA EMPRESA en Medellín, como mínimo con tres (3) días calendario de anticipación a la fecha de pago, la factura, acompañada de los siguientes documentos:
Persona natural: Fotocopia simple de su cédula de ciudadanía y una comunicación suscrita por el legítimo tenedor o por su representante legal si éste es persona jurídica, en la que se indique el nombre del banco, tipo de cuenta (ahorros, corriente u otras), número de la cuenta, NIT o cédula, teléfono y ciudad.
Persona jurídica: Original o fotocopia simple del Certificado de Existencia y Representación Legal, expedido por el organismo competente con una antelación no superior a treinta (30) días calendario, con respecto a la fecha de su presentación ante LA EMPRESA y una comunicación suscrita por el legítimo tenedor o por su representante legal si éste es persona jurídica, en la que se indique el nombre del banco, tipo de cuenta (ahorros, corriente u otras), número de la cuenta, NIT o cédula, teléfono y ciudad.
LA EMPRESA sólo recibirá la factura lunes a jueves, de 8:00 a.m. a 4:30 p.m
ISA INTERCOLOMBIA S.A. E.S.P. NIT: 900.667.590-1
Calle 12 Sur 18 - 168 Bloque 1 Pisos 2 y 3 Medellín, Colombia Tel: + 57 4 3252400
Línea única de atención: 018000 942001 Línea ética: opción 3
lineaetica@intercolombia.com
www.intercolombia.com
Docusign Envelope ID: A0E80020-8FB0-45B0-978B-B52BDC1B4E2C
isa INTERCOLOMBIA
La documentación para este trámite se radicará en LA EMPRESA y junto con el documento de aceptación correspondiente, sólo se entregará personalmente al emisor, a su Representante Legal o a su apoderado para el efecto.
7. OBLIGACIONES DE LAS PARTES.
En ejercicio de la autonomía de la voluntad y sin perjuicio de lo establecido en otras cláusulas, LAS PARTES se comprometen a cumplir las obligaciones contenidas en el Contrato y sus anexos, las que derivan de las normas jurídicas aplicables, así como las que se mencionan a continuación:
7.1. OBLIGACIONES DEL CONTRATISTA:
a. Ejecutar el objeto contractual, con la debida diligencia hasta que sus obligaciones se cumplan de conformidad y con sujeción a los términos y condiciones de este Contrato y/o dentro de los niveles de servicio establecidos.
b. Obrar con lealtad, responsabilidad y buena fe durante la ejecución del contrato.
c. Cumplir con el Código de Ética y Conducta y la Guía de Gestión y Antisoborno y Fraude.
d. Mantener vigente y actualizado durante la duración total del contrato, el registro en el Sistema de Información de Proveedores establecido por LA EMPRESA y toda la información requerida en el mismo.
e. Responder por la ejecución del contrato, para lo cual gozará de autonomía técnica y administrativa.
f. Preservar la idoneidad técnica y financiera necesaria para el cumplimiento de lo estipulado en el Contrato.
g. Comunicar a LA EMPRESA, a través de la persona que esta designe y de manera oportuna, cualquier dificultad que pueda afectar la normal ejecución del contrato. En estos casos, LA EMPRESA podrá concertar con el CONTRATISTA planes de mejoramiento e instrumentos de mitigación que permitan asegurar condiciones adecuadas para la ejecución y salvaguardar el correcto cumplimiento de lo estipulado contractualmente.
h. Emplear personal idóneo, conocedor de las labores, debidamente capacitado para la ejecución del contrato.
i. Obrar con la diligencia y el cuidado necesario para que los elementos que se requiera sean entregados o dispuestos por LA EMPRESA para la ejecución del contrato, no sufran alteración, deterioro o daño, en caso que aplique, y devolverlos a la terminación del contrato en buen estado, salvo el deterioro natural o en los términos que se acuerden para la efectiva devolución.
ISA INTERCOLOMBIA S.A. E.S.P. NIT: 900.667.590-1
Calle 12 Sur 18 - 168 Bloque 1 Pisos 2 y 3 Medellín, Colombia Tel: + 57 4 3252400
Línea única de atención: 018000 942001 Línea ética: opción 3
lineaetica@intercolombia.com
www.intercolombia.com
Docusign Envelope ID: A0E80020-8FB0-45B0-978B-B52BDC1B4E2C
isa INTERCOLOMBIA
j. Presentar los informes que sean requeridos por LA EMPRESA en virtud del objeto contractual.
k. En caso de ser procedente, garantizar que durante el tiempo de ejecución del contrato disponer de la infraestructura necesaria de hardware y software para la ejecución del objeto contractual.
l. Colaborar en forma permanente con la persona que designe LA EMPRESA, e incluso los contratistas de esta o personas asignadas por dichos contratistas (cuando aplique) en procura de lograr el cumplimiento del objeto del contrato.
m. Participar en las reuniones programadas por LA EMPRESA
n. Las demás que se establezcan en el contrato o la ley.
7.2. OBLIGACIONES DEL CONTRATANTE
a. Prestar su colaboración para el cumplimiento de las obligaciones del CONTRATISTA.
b. Obrar con lealtad, responsabilidad y buena fe durante la ejecución del contrato.
c. Suministrar la información necesaria, para la realización de la labor que se le ha encomendado.
d. Hacer los pagos a EL CONTRATISTA en los plazos contemplados en este contrato y expedir previamente la constancia de cumplimiento.
e. Formular las sugerencias por escrito sobre los asuntos que estime convenientes en el desarrollo del contrato, sin perjuicio de la autonomía propia del CONTRATISTA.
f. Cumplir estrictamente con el cronograma pactado para la ejecución de los desarrollos objeto del presente contrato.
g. Las demás que se establezcan en el contrato.
8. RENUNCIA A REQUERIMIENTOS PARA CONSTITUCIÓN EN MORA.
El CONTRATISTA renuncia expresamente a los requerimientos para la constitución en mora, por lo que no se requerirá de pronunciamiento judicial para que LA EMPRESA pueda aplicar directamente retenciones o descuentos, hacer efectiva en forma automática y de pleno derecho la cláusula denominada "Condición resolutoria expresa/ resolución de pleno derecho" del Contrato, o el cumplimiento del Contrato junto con la correspondiente indemnización de los perjuicios padecidos, en los términos aquí pactados.
9. CAUSALES DE APLICACIÓN DE LA CONDICIÓN RESOLUTORIA EXPRESA
En ejercicio de la autonomía de la voluntad, las partes acuerdan que el presente contrato podrá resolverse en forma unilateral y anticipada por parte de LA EMPRESA y de pleno derecho, en los siguientes casos:        
ISA INTERCOLOMBIA S.A. E.S.P. NIT: 900.667.590-1
Calle 12 Sur 18 - 168 Bloque 1 Pisos 2 y 3 Medellín, Colombia Tel: + 57 4 3252400
Línea única de atención: 018000 942001 Línea ética: opción 3 lineaetica@intercolombia.com
www.intercolombia.com
Docusign Envelope ID: A0E80020-8FB0-45B0-978B-B52BDC1B4E2C
isa INTERCOLOMBIA
a. Por no cumplir El Contratista los requisitos que sean necesarios para que LA EMPRESA pueda expedir la orden de inicio de la orden de entrega, dentro de los cinco (5) días hábiles siguientes a la solicitud de LA EMPRESA.
b. Por el incumplimiento del Contratista de la obligación principal que es objeto del contrato.
c. Que cualquiera de las partes, sus miembros de Junta Directiva o Junta de Socios, sus Representantes Legales o su Revisor Fiscal se encuentren en las Listas OFAC (Clinton) y ONU, en cualquier momento a partir de la adjudicación del contrato.
d. Por conductas del Contratista violatorias del Programa Empresarial de Ética y Compliance adoptado por LA EMPRESA, o cuando se compruebe que El Contratista cometió una falta que implica violación de la Declaración de Derechos Humanos, siempre que, en este último caso, medie pronunciamiento en firme de autoridad competente que así lo indique, o cuando se compruebe que El Contratista, los subcontratistas o sus proveedores, contrataron o hicieron uso de mano de obra infantil o trabajo forzado, en la ejecución de cualquier contrato de su responsabilidad.
e. Por incumplimiento de las obligaciones laborales, de seguridad social y de seguridad y salud en el trabajo por parte de cualquiera de las partes o los subcontratistas, en cualquier momento de la ejecución del contrato.
f. En caso que LA EMPRESA tenga conocimiento que los requisitos que fueron validados para el Contratista y los subcontratistas al momento de la precalificación, variaron durante la ejecución del contrato y que afecten su ejecución. En este caso, LA EMPRESA podrá solicitar la información necesaria con el fin de conocer la nueva situación del Contratista y los subcontratistas, y de considerarlo conveniente, podrá dar aplicación a la condición resolutoria expresa.
g. Por cualquier conducta del Contratista o los subcontratistas que dé lugar a que LA EMPRESA incumpla una obligación de carácter regulatorio o legal.
h. Por cualquier decisión del Contratista o los subcontratistas que ponga en riesgo o afecte la integridad física de las personas involucradas en la ejecución del contrato.
i. Por aplicación reiterada (tres -3- veces o más) de las retenciones o descuentos pactados en el contrato o por no ponerse el contratista en situación de cumplimiento por las situaciones que dieron lugar a la aplicación de dichas retenciones o descuentos.
j. En caso de que la evaluación del desempeño del Contratista no sea satisfactoria.
k. Por estar cualquiera de las partes en proceso de declaratoria de insolvencia mediante liquidación judicial.
l. Por las demás establecidas expresamente en los documentos contractuales.
PARÁGRAFO PRIMERO: Para la resolución del contrato en los términos previstos en la presente cláusula, las partes procederán a las restituciones mutuas, quedando a salvo el derecho de LA EMPRESA de aplicar la indemnización compensatoria en los términos
ISA INTERCOLOMBIA S.A. E.S.P. NIT: 900.667.590-1
Calle 12 Sur 18 - 168 Bloque 1 Pisos 2 y 3 Medellín, Colombia Tel: + 57 4 3252400
Línea única de atención: 018000 942001 Línea ética: opción 3 lineaetica@intercolombia.com
www.intercolombia.com
Docusign Envelope ID: A0E80020-8FB0-45B0-978B-B52BDC1B4E2C
isa INTERCOLOMBIA
pactados en el contrato y de perseguir en las vías judicial y extrajudicial la indemnización de los perjuicios a que hayan lugar.
PARÁGRAFO SEGUNDO: El Contratista acepta ceder el contrato a un tercero determinado por LA EMPRESA con la finalidad de que no se vea afectado el interés de ésta y sin perjuicio de la indemnización derivada de la aplicación de la condición resolutoria expresa.
10. GARANTÍAS, FIANZAS Y SEGUROS.
EL CONTRATISTA acepta incorporarse al Programa de Mitigación de Riesgos en la Contratación adoptado por LA EMPRESA en virtud del cual LA EMPRESA se constituirá como tomador, asegurado y beneficiario, de las pólizas descritas a continuación, a través de una Compañía de Seguros legalmente establecida en Colombia, a efectos de constituir al CONTRATISTA como afianzado.
EL CONTRATISTA a través del Corredor de Seguros de LA EMPRESA, hará entrega de los documentos e información requeridos para la expedición de las pólizas, por parte de la Compañía de Seguros dentro de los tres (3) días calendario siguientes contados a partir de la suscripción de cada orden de entrega. Dentro de los documentos se encuentran SARLAFT y sus anexos, presentación y experiencia de la compañía, la suscripción de la(s) contragarantía(s) necesarias, entre otros.
Las solicitudes, requerimientos y consultas podrán realizarse al personal designado por el corredor de seguros a través del correo electrónico co.seguroscontratistasisa@aon.com, o a los teléfonos 3125881811 - 3160254380 - 3142216457 - 3160174438
El CONTRATISTA se obliga a constituir a su costa y a favor de LAS EMPRESAS las siguientes garantías y seguros, los cuales deberán ser otorgados por entidades legalmente establecidas en Colombia, cuyas garantías hayan sido puestas en depósito ante la Superintendencia Financiera de Colombia, y deberán ser expedidas para entidades particulares o entre particulares o en favor de Empresas de Servicios Públicos (ESP), en forma y contenido satisfactorio para LA EMPRESA. Las garantías y seguros deberán llevar anexa la respectiva constancia de pago de la prima o el certificado que diga que la póliza no expirará por falta de pago de prima, tanto la póliza como el certificado podrán ser entregados en original, vía correo electrónico, escaneados o fotocopiados.
3.1. Garantía de Cumplimiento
EL CONTRATISTA se obliga a constituir a su costo y a favor de LA EMPRESA, una garantía de cumplimiento de la orden de entrega, expedida por una Compañía de Seguros legalmente establecida en Colombia, y cuyas garantías hayan sido puestas en depósito ante la Superintendencia Financiera de Colombia. La garantía deberá ser para entidades particulares o entre particulares o en favor de Empresas de Servicios Públicos (ESP), y deberá tener anexa la certificación de pago de la prima o el certificado que diga que la póliza no expirará por falta de pago de prima expedida por la aseguradora. Mediante esta garantía se amparan las obligaciones que contrae EL CONTRATISTA por la celebración de las
ISA INTERCOLOMBIA S.A. E.S.P. NIT: 900.667.590-1
Calle 12 Sur 18 - 168 Bloque 1 Pisos 2 y 3 Medellín, Colombia Tel: + 57 4 3252400
Línea única de atención: 018000 942001 Línea ética: opción 3
lineaetica@intercolombia.com
www.intercolombia.com
Docusign Envelope ID: A0E80020-8FB0-45B0-978B-B52BDC1B4E2C
isa INTERCOLOMBIA
ordenes de entrega, y deberá estar vigente desde la fecha de firma de cada órden por el empleado competente o por ambas partes, según sea el caso, hasta la fecha de finalización del plazo de la orden más un (1) mes. 
EL CONTRATISTA deberá presentar una garantía de cumplimiento a favor de LA EMPRESA para los servicios que ejecute y en la correspondiente orden de entrega así:
1. Para los proyectos que se desarrollen en el grupo 1: 10% del valor de cada Orden de entrega cuyo valor supere los doscientos millones de pesos (COP 200.000.000).
LA EMPRESA hará efectiva la garantía de cumplimiento cuando EL CONTRATISTA incumpla las condiciones contractuales pactadas.
Esta Garantía TIENE que constituirse dentro de los diez (10) días hábiles siguientes a la firma del contrato.
3.2. Garantía de Pago de Salarios, Prestaciones Sociales e Indemnizaciones
El CONTRATISTA debe constituir una garantía de pago de salarios, prestaciones sociales e indemnizaciones, mediante la cual responde por el pago de salarios y prestaciones sociales e indemnizaciones del personal empleado en la ejecución del objeto del Contrato y la orden de entrega. Esta garantía deberá constituirse junto con la garantía de cumplimiento y estar vigente desde la fecha de constitución de la garantía de cumplimiento hasta la fecha de finalización del plazo contractual más tres (3) años
EL CONTRATISTA deberá presentar una garantía de Garantía de Pago de Salarios, Prestaciones Sociales e Indemnizaciones a favor de LA EMPRESA para los servicios que ejecute en la correspondiente orden de entrega así:  
1. Para los proyectos que se desarrollen en el grupo 1: 10% del valor de cada Orden de entrega cuyo valor supere los doscientos millones de pesos (COP 200.000.000).
PARÁGRAFO PRIMERO: En atención a la obligación del CONTRATISTA de presentar los documentos que sean requeridos para la expedición de las pólizas, éste se hará responsable civil y penalmente por la veracidad de los mismos, y se obligará a indemnizar a LA EMPRESA, por cualquier perjuicio que se pueda derivar por la presentación de los mismos, en especial por el incumplimiento de las obligaciones contractuales derivadas del contrato de seguro, incluidas pero sin limitarse a las objeciones que la Compañía de Seguros pueda alegar por la reticencia o inexactitud en la declaración del estado real del riesgo, así como cualquier otra que le corresponda.
PARÁGRAFO SEGUNDO: La contratación de las pólizas a las que se refiere el presente numeral por parte de LA EMPRESA no exonera al CONTRATISTA de su obligación de
ISA INTERCOLOMBIA S.A. E.S.P. NIT: 900.667.590-1
Calle 12 Sur 18 - 168 Bloque 1 Pisos 2 y 3 Medellín, Colombia Tel: + 57 4 3252400
Línea única de atención: 018000 942001 Línea ética: opción 3 lineaetica@intercolombia.com
www.intercolombia.com
Docusign Envelope ID: A0E80020-8FB0-45B0-978B-B52BDC1B4E2C
isa INTERCOLOMBIA
indemnizar a ésta por todos los perjuicios derivados del incumplimiento del contrato que no fuesen cubiertos por la póliza mencionada o en exceso de esta.
PARÁGRAFO TERCERO: EL CONTRATISTA se compromete a pagar por su propia cuenta, el monto total de las primas que se causen con ocasión de la expedición de las pólizas, así como las que se causen posteriormente con motivo de modificaciones, renovaciones, prórrogas o cualquier otro anexo que se expida con fundamento en las pólizas. Dicho pago deberá hacerse por parte DEL CONTRATISTA a favor de la compañía de seguros en un solo contado, lo anterior en consideración a que la mora en el pago de la prima genera la terminación automática del contrato de seguro. EL CONTRATISTA deberá aportar el recibo de caja o comprobante de pago emitido por la compañía de seguros al Corredor de Seguros.
PARÁGRAFO CUARTO: En el evento que por capacidad financiera, listas de riesgos y/o siniestralidad con las Compañías de Seguros del Programa EL CONTRATISTA no pueda incorporarse a las pólizas tomadas por LA EMPRESA, EL CONTRATISTA se obliga y se compromete a tomar y constituir a favor de LA EMPRESA, por conducto de una Compañía de Seguros legalmente establecida en Colombia y aceptable para LA EMPRESA, unas pólizas con los mismos amparos, vigencias, condiciones generales y valores asegurados aludidos en la presente cláusula, en la que figure como beneficiario LA EMPRESA. El CONTRATISTA deberá junto con las pólizas presentar el recibo oficial del pago de la prima.
Las pólizas mencionadas en este parágrafo deberán ser constituidas de la siguiente manera:
Como condiciones particulares y/o coberturas adicionales que apliquen a todos los amparos de la póliza de Cumplimiento, se deberán incluir los siguientes:
· Inaplicabilidad de la cláusula de proporcionalidad.
· Irrevocabilidad del seguro y no modificación de los términos del seguro sin consentimiento de LA EMPRESA.
· No terminación automática del seguro.
· Amparo automático hasta de 30 días para prórrogas.
· Renuncia expresa al derecho de subrogación de la aseguradora contra LA EMPRESA, y sus filiales.
· Imposibilidad de poner excepciones de la compañía de seguros al asegurado.
· Improcedencia de la figura de la agravación del estado del riesgo, respecto al deber de informar por parte de LA EMPRESA.
· No se requerirá declaratoria judicial para demostrar el incumplimiento.
En caso de existir modificaciones, se deberá realizar la respectiva actualización o adición de cobertura.
En ningún caso los seguros anteriormente solicitados en su condicionado puedan tener como objeto el siguiente texto o un texto que exprese algo similar como se muestra a continuación:
ISA INTERCOLOMBIA S.A. E.S.P. NIT: 900.667.590-1
Calle 12 Sur 18 - 168 Bloque 1 Pisos 2 y 3 Medellín, Colombia Tel: + 57 4 3252400
Línea única de atención: 018000 942001 Línea ética: opción 3 lineaetica@intercolombia.com
www.intercolombia.com
Docusign Envelope ID: A0E80020-8FB0-45B0-978B-B52BDC1B4E2C
isa INTERCOLOMBIA
"indemnizar al asegurado original respecto de su responsabilidad legal declarada por un juez competente, mediante sentencia debidamente ejecutoriada, proveniente del desarrollo de sus operaciones en la prestación de servicios".
En las pólizas deberá contar como único Beneficiario LA EMPRESA.
PARÁGRAFO QUINTO: EL CONTRATISTA autoriza de forma expresa al CONTRATANTE a consultar, recibir y dar tramite a la información de la que trata la ley 1266 de 2008, así como validar en las centrales de riesgos, toda aquella información que considere pertinente.
PARÁGRAFO SEXTO: Las garantías y seguros de que trata esta cláusula, son accesorios al Contrato y el hecho que se contraten no limitará, alterará o afectará en forma alguna la responsabilidad civil y las obligaciones contraídas por el CONTRATISTA en virtud de este Contrato. Para las garantías y seguros que no puedan ser expedidas a través del Programa de Mitigación de Riesgos en la Contratación, no pueden ser cancelados por el Contratista sin autorización escrita de LA EMPRESA. En caso de revocación de una garantía o un seguro por parte de una compañía de seguros, el Contratista se obliga a informar a LA EMPRESA, dentro de los tres (3) días hábiles siguientes a la fecha en que le sea notificado este hecho por parte de la aseguradora. El Contratista debe mantener vigentes las garantías y seguros y serán de su cargo todas las primas y erogaciones para su constitución, prórroga o adición cuando fuere necesario, así como los deducibles aplicables en caso de siniestro.
PARÁGRAFO SÉPTIMO: Si el CONTRATISTA no cumple con los requisitos establecidos para la expedición oportuna de las garantías y seguros asociados al contrato en los términos aquí estipulados y dentro del plazo establecido para ello, LA EMPRESA podrá aplicar los términos de la Cláusula que regula la "CONDICIÓN RESOLUTORIA EXPRESA".
Las garantías y seguros expedidos por fuera del Programa de Mitigación de Riesgos en la Contratación de la EMPRESA que aquí se mencionan deben recibir visto bueno del CORREDOR DE SEGUROS del programa de mitigación de riesgos en la contratación y aprobación de LA EMPRESA, para lo cual se efectuará una revisión de las coberturas, valores, condiciones generales, condiciones particulares y constancia de pago de la prima, de acuerdo con lo establecido en el Parágrafo Cuarto.
PARÁGRAFO OCTAVO: Para contratos de plazo superior a sesenta (60) meses, el Contratista podrá presentar garantías y seguros por vigencias anuales o superiores y renovables, hasta la finalización del plazo contractual.
PARÁGRAFO NOVENO: El CONTRATISTA se obliga a informar al Corredor de seguros, cualquier modificación al contrato, así esta no implique cambio de plazo o valor.
Para las garantías y seguros que no puedan ser expedidas a través del Programa de Mitigación de Riesgos en la Contratación, si El CONTRATISTA, no cumple con mantener vigentes las garantías y seguros exigidos en el presente contrato, LA EMPRESA podrá
ISA INTERCOLOMBIA S.A. E.S.P. NIT: 900.667.590-1
Calle 12 Sur 18 - 168 Bloque 1 Pisos 2 y 3 Medellín, Colombia Tel: + 57 4 3252400
Línea única de atención: 018000 942001 Línea ética: opción 3 lineaetica@intercolombia.com
www.intercolombia.com
Docusign Envelope ID: A0E80020-8FB0-45B0-978B-B52BDC1B4E2C
isa INTERCOLOMBIA
contratar y mantener vigente cualquier garantía o seguro y pagar la prima, más no estará obligada a ello. Así mismo, podrá recuperar, en calidad de deuda incurrida por el CONTRATISTA o deducir de los pagos, dicha prima.
PARÁGRAFO DÉCIMO: Las garantías de seguros y los seguros de los contratos celebrados en pesos colombianos, se constituirán en esta moneda. Las garantías de seguros y los seguros de los contratos celebrados en monedas diferentes al peso colombiano se constituirán en dólares de los Estados Unidos.
11. CONDICIÓN RESOLUTORIA EXPRESA/ RESOLUCIÓN DE PLENO DERECHO.
En ejercicio de la autonomía de la voluntad las partes acuerdan que en caso de que el CONTRATISTA no cumpla la obligación principal contraída con ocasión del perfeccionamiento del contrato, LA EMPRESA en su condición de contratante cumplido, estará facultada para resolver anticipadamente el presente negocio jurídico, en forma automática y de pleno derecho.
En caso de incumplimiento por parte de LA EMPRESA el CONTRATISTA podrá aplicar la Condición Resolutoria Tácita, en los términos definidos por la legislación aplicable. En todo caso, el Contratista deberá efectuar la entrega o devolución de los bienes que sean de propiedad de La Empresa, así como las obras que haya estado ejecutando con ocasión del Acuerdo de Voluntades, hasta el momento de aplicación de la Condición Resolutoria.
12. RECOMPOSICIÓN DEL CONTRATO.
Si durante la ejecución del contrato se hiciera recurrente la aplicación de retenciones o descuentos, LA EMPRESA podrá optar por: (a) exigir al CONTRATISTA el cumplimiento del mismo, procediendo las partes a recomponerlo en lo que resulte necesario para asegurar el cumplimiento de las obligaciones contraídas por el CONTRATISTA; (b) acordar la cesión de la posición contractual del CONTRATISTA en favor de un tercero aprobado por LA EMPRESA, cuando el CONTRATISTA no se encuentre en condiciones que le permitan terminar de ejecutar cabalmente sus obligaciones; en ambos casos, con la correspondiente indemnización del perjuicios en favor de LA EMPRESA.
13. DERECHO DE CONTRADICCIÓN.
Para garantizar el derecho de contradicción y mantener a salvo la equivalencia prestacional, las partes acuerdan agotar el siguiente conducto regular para la aplicación de retenciones o descuentos previstas en la órdenes de entrega, resolución, terminación anticipada del contrato y devoluciones por eximentes de responsabilidad:
a. LA EMPRESA comunicará por escrito al CONTRATISTA su intención de aplicar retenciones o descuentos, resolver o terminar anticipadamente el contrato, y la causal que lo justifica. b. Recibida la comunicación, el CONTRATISTA contará con tres (3) días hábiles
ISA INTERCOLOMBIA S.A. E.S.P. NIT: 900.667.590-1
Calle 12 Sur 18 - 168 Bloque 1 Pisos 2 y 3 Medellín, Colombia Tel: + 57 4 3252400
Línea única de atención: 018000 942001 Línea ética: opción 3
lineaetica@intercolombia.com
www.intercolombia.com
Docusign Envelope ID: A0E80020-8FB0-45B0-978B-B52BDC1B4E2C
isa INTERCOLOMBIA
para explicar por escrito los motivos eximentes de su responsabilidad en relación con la causal invocada por LA EMPRESA y explicará las soluciones que propone para garantizar el cumplimiento inmediato de sus obligaciones. c. LA EMPRESA analizará la propuesta de cumplimiento que le formule el CONTRATISTA, determinará y le informará por escrito, si continúa con la ejecución del Contrato y en qué términos, si procede a la aplicación de la retención o descuento, según el caso; o si decide hacer efectiva la resolución o la terminación anticipada del mismo, según aplique en los términos pactados, debiendo en este último caso, procederse al cruce de cuentas del Contrato. d. Si posterior a la aplicación de retenciones o descuentos, el CONTRATISTA acredita la existencia de situaciones que lo exoneren de responsabilidad, LA EMPRESA le devolverá los dineros deducidos, debidamente actualizados mediante la aplicación del IPC certificado por el DANE o quién haga sus veces, acumulado anualmente, entre la fecha de la deducción y la de la devolución, para el caso de empresas domiciliadas en Colombia. Para las empresas domiciliadas por fuera de Colombia se determinará el indicador o condición de ajuste de los dineros retenidos al momento de realizar la devolución.
14. SOLUCIÓN DE CONTROVERSIAS.
Las controversias que se susciten en la ejecución de las condiciones generales del presente contrato o de las órdenes de entrega cuyo domicilio o normatividad aplicable sea la República de Colombia, deberán ser resueltas mediante acuerdo directo entre las partes, el cual se adelantará en un término máximo de veinte (20) días hábiles, contados desde el momento en que una parte informa a la otra la existencia de una controversia, y la solución acordada a la misma constará en un Acuerdo Formal de Transacción suscrito por ambas partes. De no lograrse acuerdo entre las partes, la diferencia se resolverá en primera instancia, por el mecanismo de Conciliación prejudicial y de no haber acuerdo, se ejercerá la acción contractual ante la Jurisdicción Contencioso Administrativa.
15. CONFIDENCIALIDAD Y PROTECCIÓN DE DATOS PERSONALES.
LAS PARTES se obligan a cumplir a cabalidad con lo dispuesto en las normas sobre protección de datos personales. Así mismo, cada una de LAS PARTES se obliga a guardar reserva de la información que le sea entregada por la otra parte, o a la que tenga acceso en desarrollo del presente Contrato, con independencia del medio en el cual se encuentre soportada. En cumplimiento de la obligación de reserva cada parte deberá: a. Utilizar la información suministrada por la otra parte para la labor encomendada; comprometerse, además, a utilizar todos los medios a su alcance para garantizar la más estricta confidencialidad respecto de dicha información, y hacer extensivo dicho deber de confidencialidad a cualquier persona que por su relación con él, deba tener acceso a dicha información, para el correcto cumplimiento de sus obligaciones. b. Abstenerse de reproducir, modificar, hacer pública o divulgar a terceros la información objeto del presente compromiso, sin previa autorización escrita de la parte que se la reveló. LAS PARTES reconocen que la información entregada por la otra parte le pertenece a esta o al tercero que haya autorizado a la parte receptora para suministrar la información confidencial. Por ello, se obligan mutuamente a resarcir los perjuicios por daño emergente y/o lucro cesante que llegare a ocasionar la revelación, divulgación o utilización realizada de cualquier forma
ISA INTERCOLOMBIA S.A. E.S.P. NIT: 900.667.590-1
Calle 12 Sur 18 - 168 Bloque 1 Pisos 2 y 3 Medellín, Colombia Tel: + 57 4 3252400
Línea única de atención: 018000 942001 Línea ética: opción 3
lineaetica@intercolombia.com
www.intercolombia.com
Docusign Envelope ID: A0E80020-8FB0-45B0-978B-B52BDC1B4E2C
isa INTERCOLOMBIA
distinta a la definida en el presente Contrato, ya sea de forma dolosa o culposa. El presente compromiso de confidencialidad tendrá vigencia desde la fecha de la firma del Contrato y hasta cinco (5) años más después de la terminación del mismo.
Se podrá revelar o divulgar la Información Confidencial en los siguientes eventos: a. En desarrollo o por mandato de una ley, decreto, sentencia u orden de autoridad competente en ejercicio de sus funciones legales. En este caso, la Parte receptora se obliga a avisar inmediatamente haya tenido conocimiento de esta obligación a la Parte reveladora, para que pueda tomar las medidas necesarias para proteger la Información Confidencial, y de igual manera se compromete a tomar las medidas necesarias para atenuar los efectos de tal divulgación y se limitará a divulgar únicamente la información efectivamente requerida por la autoridad competente; b. Cuando la Información Confidencial este o llegue a estar a disposición del público o sea de dominio público por causa diferente a un acto u omisión de la parte receptora; c. Cuando la Información Confidencial haya estado en posesión de la Parte receptora antes de que hubiese recibido la misma por medio de la parte reveladora o que no hubiese sido adquirida de la parte reveladora, o de cualquier tercero que tuviere un compromiso de confidencialidad con respecto a la parte reveladora.
En relación con los datos personales que deba suministrar con ocasión del proceso de contratación y el contrato que surja con ocasión de la adjudicación, autorizo para su tratamiento conforme las políticas que tienen Las Partes, lo que incluye el almacenamiento de datos a nivel nacional o internacional y en especial para la finalidad de ser contactado y/o notificado de cualquier información de interés de Las Partes; así mismo declaro que en el caso que me deban suministrar datos personales que se requieran para la ejecución del contrato que surja con ocasión de la adjudicación, entiendo y acepto que en calidad de encargado cumpliré las políticas de tratamiento de datos personales de Las Partes y no usaré dichos datos para fines comerciales propios o de terceros, esto es, con el fin de lograr una protección efectiva del Derecho Fundamental de Habeas Data, El Proponente se compromete a: i) No hacer ningún uso o difusión de los datos personales suministrados para ejecutar el contrato, sin contar con la debida autorización del titular de dichos datos. Dicha obligación subsistirá aún finalizada la relación contractual. ii) Conservar la información bajo las condiciones de seguridad necesarias para impedir su adulteración, pérdida, consulta, uso o acceso no autorizado o fraudulento.
16. PROPIEDAD INTELECTUAL Y DERECHOS DE AUTOR.
EL CONTRATISTA declara que es titular o que tiene los derechos de explotación de la propiedad intelectual y/o derechos patrimoniales de autor que están inmersos en la ejecución del contrato que surja con ocasión de la adjudicación. Igualmente, LA EMPRESA no autoriza al uso de sus elementos de propiedad intelectual ni a la explotación de los derechos patrimoniales de autor del que sean titulares con la sola adjudicación, para lo cual se requerirá autorización escrita. En caso de que la ejecución conlleve una prestación de servicios, se entiende que todos los resultados, obras, signos distintivos, nuevas creaciones y en general las creaciones elaborados en virtud o con ocasión del contrato que se perfecciona con la adjudicación serán de propiedad y titularidad exclusiva de LA EMPRESA.
ISA INTERCOLOMBIA S.A. E.S.P. NIT: 900.667.590-1
Calle 12 Sur 18 - 168 Bloque 1 Pisos 2 y 3 Medellín, Colombia Tel: + 57 4 3252400
Línea única de atención: 018000 942001 Línea ética: opción 3
lineaetica@intercolombia.com
www.intercolombia.com
Docusign Envelope ID: A0E80020-8FB0-45B0-978B-B52BDC1B4E2C
isa INTERCOLOMBIA
17. EXIMENTE DE RESPONSABILIDAD / FUERZA MAYOR.
El CONTRATISTA quedará exento de toda responsabilidad por cualquier daño o demora en la ejecución del Contrato, como resultado de causas constitutivas de fuerza mayor, caso fortuito o hechos de terceros debidamente comprobados, casos en los cuales tendrá derecho a la modificación de los plazos, pero no al reconocimiento de indemnización alguna por parte de la EMPRESA. Dentro de las setenta y dos (72) horas siguientes al momento en que el CONTRATISTA se vea obligado a suspender la ejecución del Contrato debido a la ocurrencia de un hecho constitutivo de fuerza mayor, caso fortuito o hechos de terceros, deberá comunicar el hecho a la EMPRESA, señalando las causas que obligan a la suspensión. Una vez comprobado el hecho, la EMPRESA podrá tomar alguna o algunas de las siguientes alternativas: a. Ampliar el plazo del Contrato. b. Disminuir o ampliar el alcance del Contrato. c. Acordar con el CONTRATISTA la terminación bilateral anticipada del Contrato.
18. LISTAS DE VIGILANCIA INTERNACIONALES.
LAS PARTES declara que no se encuentra en las Listas OFAC (Clinton) y ONU; así mismo, se responsabiliza ante la EMPRESA porque sus miembros de Junta Directiva o Junta de Socios, sus Representantes Legales o su Revisor Fiscal, tampoco se encuentren en dichas Listas. El CONTRATISTA acepta que la inobservancia de la presente disposición podrá dar lugar a la resolución del contrato en los términos de la Cláusula que regula la "CONDICIÓN RESOLUTORIA EXPRESA" de este Contrato.
19. CAPACIDAD TÉCNICA.
El CONTRATISTA aplicará la capacidad técnica y administrativa que sea indispensable para la correcta y eficiente ejecución del objeto del Contrato, pudiendo LA EMPRESA exigir que se apliquen los correctivos necesarios para el cabal cumplimiento del mismo. El CONTRATISTA debe garantizar la idoneidad del personal con el cual va a ejecutar los trabajos contratados y velar por su permanente formación.
20. CAMBIOS.
Toda adición, modificación o reforma que surja con ocasión del contrato deberá realizarse mediante la suscripción de una cláusula adicional. El retardo o la omisión de las partes en hacer efectivo cualquier derecho, facultad o privilegio no tendrá el efecto de una renuncia, a dicho derecho, facultad o privilegio.
21. CLÁUSULA ANTICORRUPCIÓN
LAS PARTES declaran y garantizan que:
1. Cumplirán toda la normatividad que le sea aplicable, así como los estándares a los que voluntariamente se haya adherido y a su vez con los requerimientos y órdenes de las autoridades que sean competentes, nacionales e internacionales, que tengan por objeto o por efecto la prevención, control y administración del riesgo de fraude, soborno y corrupción; teniendo implementados con eficiencia las políticas y procedimientos de cumplimiento que se derivan de dichas disposiciones legales y en especial observará las normas relacionadas con estas materias que se encuentren en otras disposiciones del Contrato;
ISA INTERCOLOMBIA S.A. E.S.P. NIT: 900.667.590-1
Calle 12 Sur 18 - 168 Bloque 1 Pisos 2 y 3 Medellín, Colombia Tel: + 57 4 3252400
Línea única de atención: 018000 942001 Línea ética: opción 3 lineaetica@intercolombia.com
www.intercolombia.com
Docusign Envelope ID: A0E80020-8FB0-45B0-978B-B52BDC1B4E2C
isa INTERCOLOMBIA
2. Mantendrán en cualquier relación con un representante de la administración pública en el desempeño de su actividad, una relación guiada por el respeto institucional y la transparencia;
3. No incurrirán en acciones relacionadas con fraude, soborno y/o corrupción, en relación con funcionarios públicos, candidatos para un servicio público o con particulares, como tampoco realizarán actividades ilícitas en cualquier país donde opera, ni poseen litigios, demandas o investigaciones en curso en dichos territorios que puedan afectar la celebración y ejecución del Contrato de forma alguna;
4. No usará a la otra parte ni sus empresas o la relación con éstas, o cualquier otro tercero, para cometer actos ilícitos o relacionados con fraude, soborno y/o corrupción; en especial, para obtener un negocio con la otra parte y sus empresas o para alterar los términos, condiciones o cumplimiento de cualquier contrato o pedido de la otra parte y sus empresas, incluido, entre otros, este Contrato;
5. Ninguna de las partes, ni sus filiales, están directa o indirectamente participadas o controladas, total o parcialmente, por un funcionario público susceptible de tomar o influir en actuaciones oficiales a favor o en contra de cualquiera de las Partes o del Contrato, y ningún administrador de hecho o de derecho o empleado de cualquiera de las partes con capacidad para influir en las decisiones que afectan el Contrato se espera que tenga la condición de funcionario público a lo largo de la ejecución del Contrato, no obstante, si esto último ocurriera, se notificará de forma inmediata;
6. Se abstendrán de tener vínculos con terceros que se conozca por cualquier medio estén vinculados con actividades de fraude, soborno y/o corrupción; por lo que implementara medidas para administrar con seguridad razonable los riesgos de fraude, soborno y/o corrupción de terceros y en caso de que actúe como mandatario de LA EMPRESA, deberá cumplir los procedimientos que señale ésta para la prevención de este tipo de riesgos;   
7. Iniciarán las acciones legales procedentes, frente a los hechos fraude, soborno y/o corrupción, absteniéndose de iniciar acciones legales en nombre o representación de la otra parte;
8. Participarán en las actividades de divulgación que establezcan las partes para la prevención los riesgos de fraude, soborno y/o corrupción y adoptará los protocolos que de dichas actividades se deriven o que sean desarrollo de lo establecido en esta sección y que se entenderán incorporados al Contrato; para lo cual se comprometen a exigir la presencia de los representantes involucrados con la ejecución del Contrato y garantizarán que todos los que actúen a nombre de cualquiera de las partes o con recursos de este contrato, conozcan y cumplan las disposiciones de esta sección;
9. Mantendrán registros precisos de todas las transacciones relativas a este Contrato de acuerdo con las prácticas de contabilidad generalmente aceptadas y en cumplimiento de las demás normas aplicables de forma que permitan identificar debidamente las distintas erogaciones;
10. Informarán a otra parte en caso de tener conocimiento o sospecha acerca de cualquier riesgo de fraude, soborno y/o corrupción o de la mención en cualquier proceso
ISA INTERCOLOMBIA S.A. E.S.P. NIT: 900.667.590-1
Calle 12 Sur 18 - 168 Bloque 1 Pisos 2 y 3 Medellín, Colombia Tel: + 57 4 3252400
Línea única de atención: 018000 942001 Línea ética: opción 3 lineaetica@intercolombia.com
www.intercolombia.com
Docusign Envelope ID: A0E80020-8FB0-45B0-978B-B52BDC1B4E2C
isa INTERCOLOMBIA
administrativo o judicial de cualquier hecho relacionado con los anteriores riesgos a fin que la otra parte pueda tomar las medidas de administración pertinentes, o de cualquier hecho que pudiera implicar el incumplimiento de cualquier declaración o garantía señaladas en esta sección, o de cualquier duda en relación con la aplicación de lo establecido en esta sección y colaborará en las investigaciones a que haya lugar y entregará la información que ésta requiera para la prevención de este tipo de riesgos. En todo caso, las partes siempre actuarán de manera que se mitigue en todo caso la agravación de los riesgos de fraude, soborno y/o corrupción.
11. £ Las anteriores declaraciones se extienden a los empleados y administradores de hecho o de derecho de las partes, así como a los socios con una participación superior al cinco por ciento (5%) y también a los terceros en general de las partes que tengan una influencia en éste.
Durante el periodo de vigencia del Contrato y durante cinco (5) años después de terminación de éste y en todo caso dentro del plazo máximo de prescripción de los delitos relacionados con fraude, soborno y/o corrupción de acuerdo con la normatividad aplicables, las partes aceptan mantener todos los registros y libros de cuentas habituales y adecuados y todas las anotaciones habituales y correctas relacionadas con sus costos y gastos relacionados con el Contrato, por lo que deberá exigir a sus representantes, subcontratistas, prestadores de servicios, proveedores y personal de estos que hagan lo propio. Durante el periodo descrito anteriormente, las partes podrán auditar y/o inspeccionar los libros, registros e instalaciones para verificar el cumplimiento de las declaraciones contenidas en esta sección y de las facturas emitidas por EL CONTRATISTA, así como la de los representantes, subcontratistas, prestadores de servicios, proveedores y personal de estos, por lo que LA EMPRESA podrá exigirle a EL CONTRATISTA que incluya una cláusula en las relaciones con estos últimos que facilite el cumplimiento de esta obligación. Las auditorías y/o inspecciones las realizará el personal de cada parte o las personas que estos designen. EL CONTRATISTA acepta proporcionar al equipo de auditoría o inspección designado por LA EMPRESA acceso razonable a los registros e instalaciones de EL CONTRATISTA. Con ocasión de la auditoría y/o inspección, se podrá acordar un plan de mejoramiento, sin perjuicio de la consideración en los programas de evaluación de desempeño de EL CONTRATISTA. LA EMPRESA asumirá el gasto de su equipo de inspección o sus auditores, sin embargo, se podrá solicitar reembolso de dichos gastos a        
EL CONTRATISTA en caso de incumplimiento de las declaraciones contenidas en esta sección. En todo caso, antes de que se pueda proceder a la auditoría, la parte que la vaya a realizar deberá notificar a la parte auditada con una antelación de por lo menos cinco (5) días hábiles a la realización de la misma, así como también acompañar dicha notificación de la lista de chequeo de los temas a auditar, los cuales, en todo caso, solo podrán ser sobre lo relacionado específicamente al presente Contrato.
Además, en caso de incumplimiento de las declaraciones de esta sección o del plan de mejoramiento o la realización de actos que impidan materialmente realizar la inspección o auditoría y de cualquier investigación en general y/o de la imposición de cualquier sanción contra EL CONTRATISTA derivada del desconocimiento de la normatividad de fraude, soborno y corrupción que le sea aplicable y/o la inclusión en las listas restrictivas, vinculantes o sancionatorias, nacionales o extranjeras, relacionadas con la prevención de este tipo de riesgo o de cualquier otro riesgo que pueda derivar en fraude, soborno y/o
ISA INTERCOLOMBIA S.A. E.S.P. NIT: 900.667.590-1
Calle 12 Sur 18 - 168 Bloque 1 Pisos 2 y 3 Medellín, Colombia Tel: + 57 4 3252400
Línea única de atención: 018000 942001 Línea ética: opción 3
lineaetica@intercolombia.com
www.intercolombia.com
Docusign Envelope ID: A0E80020-8FB0-45B0-978B-B52BDC1B4E2C
isa INTERCOLOMBIA
corrupción, incluidas las listas no vinculantes para LA EMPRESA, en especial, en la lista de empresas y personas sancionadas por el Banco Mundial por haberse determinado que estuvieron involucradas en prácticas fraudulentas, corruptas, colusorias, coercitivas u obstructivas; LA EMPRESA podrá resolver o terminar el Contrato o acordarse una suspensión de este, o solicitar la nulidad del Contrato por vicios de consentimiento de acuerdo con las declaraciones contenidas en esta sección, sin que EL CONTRATISTA tenga derecho a ningún pago adicional por concepto de daños o de indemnización, al mismo tiempo que los pagos contractualmente debidos sólo podrán pagarse en la medida en que lo permita la ley aplicable y/o los procedimientos y/o controles que tenga implementada LA EMPRESA.
Si LA EMPRESA decide hacer una suspensión de los pagos, se entiende autorizado desde ahora por EL CONTRATISTA. En el evento de resolución o terminación, se aplicará lo establecido en el derecho de contradicción y en este caso EL CONTRATISTA deberá documentar ante LA EMPRESA que ha implementado eficazmente medidas para prevenir el riesgo de fraude, soborno y/o corrupción. Finalmente, se entiende autorizada LA EMPRESA para revelar información de EL CONTRATISTA ante cualquier autoridad relacionada con los hechos de fraude, soborno y corrupción, sin perjuicio de los procedimientos del canal de denuncias que tenga establecidos LA EMPRESA. Una vez terminado el Contrato, LA EMPRESA podrá considerar este hecho como un impedimento para la aceptación de solicitudes de oferta, pedidos, órdenes y en general el perfeccionamiento y ejecución de cualquier relación comercial.
Si hubiere recaído sentencia condenatoria contra los administradores de hecho o de derecho o empleados de EL CONTRATISTA que gestionan directa o indirectamente el Contrato, en cualquier proceso administrativo o judicial relacionado con hechos que impliquen riesgos de fraude, soborno y/o corrupción; o que estén incluidos en las listas restrictivas, vinculantes o sancionatorias, nacionales o extranjeras, relacionadas con la prevención de este tipo de riesgo o de cualquier otro riesgo que pueda derivar en fraude, soborno y/o corrupción, incluidas las listas no vinculantes para LA EMPRESA; podrá esta última solicitar a EL CONTRATISTA, la remoción de los administradores de hecho o de derecho o empleados condenados o incluidos en las listas mencionadas. Lo anterior, sin perjuicio que LA EMPRESA podrá resolver o terminar el Contrato o acordarse una suspensión de este, o solicitar la nulidad del Contrato por vicios de consentimiento de acuerdo con las declaraciones contenidas en esta sección, sin que EL CONTRATISTA tenga derecho a ningún pago adicional por concepto de daños o de indemnización.
Ninguna instrucción recibida de LA EMPRESA podrá ser interpretada como destinada a autorizarlo para cometer o participar en cualquier hecho constitutivo de delito, y en especial, de riesgo de fraude, soborno y/o corrupción.
Cualquier entidad, que se pronuncie de conformidad con las disposiciones de resolución de controversias del Contrato, estará facultada para determinar las consecuencias contractuales de cualquier presunto incumplimiento de esta sección.
En todo caso, EL CONTRATISTA será responsable ante LA EMPRESA, sus accionistas, administradores o empleados y las contrapartes relacionadas con el Contrato, frente a
ISA INTERCOLOMBIA S.A. E.S.P. NIT: 900.667.590-1
Calle 12 Sur 18 - 168 Bloque 1 Pisos 2 y 3 Medellín, Colombia Tel: + 57 4 3252400
Línea única de atención: 018000 942001 Línea ética: opción 3 lineaetica@intercolombia.com
www.intercolombia.com
Docusign Envelope ID: A0E80020-8FB0-45B0-978B-B52BDC1B4E2C
isa INTERCOLOMBIA
cualquier demanda, pérdida de daños y perjuicios derivados o relacionados con el incumplimiento de las declaraciones contenidas en esta sección o con los supuestos descritos en la misma, los cuales en todo caso, estarán limitados al valor total del contrato, excepto por aquellos casos que resulten de culpa grave o dolo del Contratista, lo cual será determinado por sentencia judicial o laudo arbitral ejecutoriado. En ningún caso, el Contratista sera responsables por daños consecuenciales, especiales, indirectos, incidentales, punitivos o ejemplares (incluyendo, sin que implique limitación, perjuicios y costos de oportunidad). Las disposiciones de esta cláusula aplicarán independientemente de la fuente de la obligación y la naturaleza de la acción que fuere aplicable.
Los hechos y actos de fraude, soborno y/o corrupción a los que se refiere esta sección se interpretará de acuerdo con las leyes que le aplican a LA EMPRESA y conforme la Guía Gestión Anticorrupción y Antisoborno de ISA y sus Empresas, la cual hará parte integral del presente Contrato, así como las demás cláusulas establecidas en éste, en el cual se pueden establecer obligaciones particulares de acuerdo con las leyes que le aplican a LA EMPRESA. Ante cualquier dificultad de interpretación se entiende por fraude: cualquier acto u omisión, incluida la tergiversación de hechos y circunstancias, que deliberada o imprudentemente engañen, o intenten engañar, a alguna parte para obtener un beneficio financiero o de otra naturaleza o para evadir una obligación; y por soborno y corrupción: ofrecer, prometer, entregar, aceptar o solicitar una ventaja inapropiada de cualquier valor (de naturaleza financiera o no) directa o indirectamente, e independiente de su ubicación, para influenciar indebidamente las acciones de la otra parte o como incentivo o recompensa para que una persona actúe o deje de actuar en relación con el desempeño de las obligaciones de estas personas."
22. DESCUENTO COMPENSATORIO.
En ejercicio de la autonomía de la voluntad las partes acuerdan que en caso de materialización de la Condición Resolutoria Expresa en los términos indicados en el presente documento, El CONTRATISTA deberá cancelar en favor de LA EMPRESA, como tasación anticipada de perjuicios, una suma equivalente al diez por ciento (10%) del valor total acordado, lo que la EMPRESA podrá hacer efectivo mediante la ejecución de la carta fianza otorgada por el CONTRATISTA para tal fin, la cual será considerada como pago parcial, pero no definitivo de los perjuicios causados a LA EMPRESA sin perjuicio del derecho de LA EMPRESA, de perseguir la indemnización plena de perjuicios por la vía prejudicial o judicial. Así mismo, las partes convienen que el valor acordado como descuento compensatorio podrá ser deducido directamente por LA EMPRESA de los dineros que adeude al CONTRATISTA, en caso de haberlos, o deberá ser girado por El CONTRATISTA a la cuenta bancaria que LA EMPRESA le indique, en caso de no haber saldos pendientes de pago.
PARÁGRAFO PRIMERO: (NOTA: Este parágrafo solo se incluye cuando el contrato incluya la cláusula de RETENCIONES Y DESCUENTOS) Los valores retenidos (o descontados, según el caso) por LA EMPRESA al CONTRATISTA a título de (RETENCIÓN O DESCUENTO, según el caso), se imputarán como abono a la suma que constituya el descuento compensatorio.
ISA INTERCOLOMBIA S.A. E.S.P. NIT: 900.667.590-1
Calle 12 Sur 18 - 168 Bloque 1 Pisos 2 y 3 Medellín, Colombia Tel: + 57 4 3252400
Línea única de atención: 018000 942001 Línea ética: opción 3
lineaetica@intercolombia.com
www.intercolombia.com
Docusign Envelope ID: A0E80020-8FB0-45B0-978B-B52BDC1B4E2C
isa INTERCOLOMBIA
PARÁGRAFO SEGUNDO: El CONTRATISTA manifiesta expresamente que faculta a LA EMPRESA para que retenga las sumas a su favor y las compense contra el valor de presente cláusula, en caso de que se llegara a hacer exigible.
23. RESPONSABILIDAD.
La responsabilidad del Contratista no estará limitada. En todo caso, no habrá limitación a la responsabilidad en caso de dolo o negligencia grave por parte del Contratista o por los procesos sancionatorios administrativos, sin limitarse a los ambientales o procesos penales; así como la limitación no se extiende a los daños causados por muerte o lesiones personales, a las reclamaciones de terceros referidas en la cláusula de indemnidad y por conductas culposas del Contratista que ocasionen sanciones a la Empresa en virtud de la ejecución del Contrato.
24. INDEMNIDAD DE LA EMPRESA.
Cada una de las Partes mantendrá indemne a la otra, a sus empleados, agentes y representantes:
1. Contra toda reclamación, demanda, acción legal, proceso penal, sanción judicial o administrativa o costos que surjan o se le causen, por daños a los bienes y/o lesiones a terceros, al personal de las Partes y, por razones imputables a una de ellas. Se consideran como imputables al Contratista todas las acciones u omisiones del personal a su servicio o bajo su dependencia.
2. Por todas las reclamaciones o demandas laborales presentadas por acreedores o trabajadores de sus contratistas y subcontratistas.
3. Por razón de cualquier reclamación, demanda, acción legal o costos que surjan o se le causen como resultado del uso por una Parte, de patentes, diseños, derechos de autor que sean propiedad de terceros, durante el desarrollo del Contrato o con posterioridad al mismo.
4. Por sanciones por parte de las autoridades ambientales atribuibles al Contratista y/o a su subcontratista y en contra de la Empresa por incumplimiento de las obligaciones legales ambientales en el marco de las actividades objeto del Contrato.
5. Por las demás circunstancias señaladas en el Contrato.
En caso de que se entable una reclamación, requerimiento, demanda o acción legal, éste será comunicado en un término máximo de setenta y dos (72) horas contados a partir del recibo de la solicitud, para que la otra Parte por su cuenta adopte oportunamente las medidas para mantener indemne a la Parte que recibe la solicitud.
Por lo tanto, LAS PARTES adelantaran las gestiones a fin de solucionar las diferencias. Adicionalmente, la parte incumplida deberá salir a la defensa de la parte cumplida y asumir todos los gastos, incluidos los de honorarios de abogados y peritos, de suerte que ante la ausencia de defensa o ante una diferencia técnica en la defensa, podrá realizarlo directamente la parte cumplida y la parte incumplida deberá pagar los gastos en los que haya incurrido la parte cumplida, sin lugar a requerimiento adicional alguno. Igualmente, en virtud de la indemnidad, Las PARTES tendrán derecho de vincular a la otra PARTE como tercero en los procesos judiciales a que haya lugar en virtud del objeto de este contrato. Lo anterior, sin perjuicio de las demás obligaciones de indemnidad establecidas en este Contrato.
ISA INTERCOLOMBIA S.A. E.S.P. NIT: 900.667.590-1
Calle 12 Sur 18 - 168 Bloque 1 Pisos 2 y 3 Medellín, Colombia Tel: + 57 4 3252400
Línea única de atención: 018000 942001 Línea ética: opción 3
lineaetica@intercolombia.com
www.intercolombia.com
Docusign Envelope ID: A0E80020-8FB0-45B0-978B-B52BDC1B4E2C
isa INTERCOLOMBIA
En caso de que alguna de las partes sea condenada u objeto de cualquier sanción pecuniaria, compensaciones relacionadas con normas técnicas de calidad en caso que aplique o de otra índole por parte de las autoridades competentes, la parte incumplida deberá reembolsar de cualquier otra forma compensar por dicha sanción o compensación (esta compensación incluirá pero no estará limitada a honorarios de abogados y/o de consultores que la Empresa hubiese tenido que emplear o contratar a fin de ejercer su defensa). En todo caso, la PARTE cumplida podrá repetir contra la parte incumplida por las sumas que tenga que pagar en virtud de la condena y cobrar además cualquier perjuicio adicional que le sea causado.
Asimismo, en el caso que alguna de las PARTES se viera involucrada en alguna acción de investigación, supervisión y/o fiscalización administrativa, policial, fiscal o judicial, o en procedimientos de naturaleza laboral, penal, ambiental, de seguridad social o de cualquier otra índole derivados de las actividades desarrolladas por la otra parte en virtud del presente Contrato y que sean notificados de manera posterior al término del Contrato, sin perjuicio de la causa de su terminación; LA PARTE INCUMPLIDA asumirá sus responsabilidades y realizará los actos necesarios para librar de responsabilidad a la otra PARTE, así mismo reconocerá y reembolsará todo gasto que este último hubiera asumido con ocasión de estas circunstancias. Esta obligación subsistirá con posterioridad a la fecha de terminación de la relación contractual entre las Partes sin perjuicio del motivo de la terminación.
25. NORMAS DE EXTRANJERÍA.
El CONTRATISTA como contratante o empleador directo del personal extranjero que utilice para el desarrollo de las actividades objeto del presente contrato, verificará que este personal cuente con las autorizaciones requeridas por la Agencia Nacional de Migración - Migración Colombia- o la autoridad que haga sus veces, con el fin de que dicho personal esté habilitado para desarrollar el objeto contractual. La omisión de estos requisitos lo hará responsable de las sanciones respectivas ante las autoridades colombianas competentes.
26. IMPUESTOS.
Las retenciones en la fuente a que hubiere lugar y todo impuesto, tasa o contribución directa o indirecta, Nacional, Departamental o Municipal que se cause por razón de la celebración, ejecución y pago de este Pedido serán a cargo exclusivo del sujeto pasivo que la ley defina. El Contratista discriminará el Impuesto sobre las Ventas (IVA) en cada factura cuando dicho Impuesto sea aplicable y LA EMPRESA se lo pagará de acuerdo con las normas y tarifas vigentes para el mismo en el momento de su causación. La factura debe contener los requisitos establecidos en el Artículo 617 del Estatuto Tributario. El Contratista declara conocer las normas tributarias colombianas vigentes, aplicables.
27. ADMINISTRACIÓN DEL CONTRATO.
LA EMPRESA realizará control a la ejecución del presente Contrato, haciendo seguimiento técnico, administrativo, jurídico, financiero y contable al cumplimiento de sus propias obligaciones y de las adquiridas por el CONTRATISTA mediante la suscripción del mismo.
ISA INTERCOLOMBIA S.A. E.S.P. NIT: 900.667.590-1
Calle 12 Sur 18 - 168 Bloque 1 Pisos 2 y 3 Medellín, Colombia Tel: + 57 4 3252400
Línea única de atención: 018000 942001 Línea ética: opción 3
lineaetica@intercolombia.com
www.intercolombia.com
Docusign Envelope ID: A0E80020-8FB0-45B0-978B-B52BDC1B4E2C
isa INTERCOLOMBIA
La administración del contrato se hará por la EMPRESA o terceros designados por ésta para tal efecto. Los defectos en la administración o falta de esta, no atenuarán ni eximirán al CONTRATISTA frente al cumplimiento del objeto contratado.
28. SUSPENSIÓN TEMPORAL DEL CONTRATO.
En ejercicio de la autonomía de la voluntad o por circunstancias de fuerza mayor, caso fortuito o hechos de terceros, las Partes de común acuerdo pueden suspender la ejecución del Contrato, suscribiendo un acta de suspensión, en donde conste el evento y se acuerden las condiciones de la misma. Para efectos del plazo del Contrato, no se tendrá en cuenta el tiempo de suspensión y se deberán actualizar las pólizas. Para reanudar la ejecución del Contrato, las Partes suscribirán un acta de reanudación, en la que se indicará la forma en que deberá contarse el plazo a partir del reinicio de las actividades objeto del Contrato.
29. CESIÓN.
Ni la oferta, ni el contrato que surja con ocasión de la oferta podrá ser cedido total o parcialmente por cualquiera de las partes, sin previa autorización expresa y escrita de la otra parte.
30. MÉRITO EJECUTIVO.
El contrato de las partes integrado por medio de la solicitud, la oferta y la comunicación de adjudicación presta mérito ejecutivo para todos los efectos legales. En consecuencia, para hacer efectivos todos los derechos y obligaciones en el contenidos y en los términos aquí pactados, bastará con la presentación de los documentos que lo integran.
31. DOCUMENTOS.
Regulan, complementan, adicionan y determinan las condiciones del presente Contrato, los documentos que se citan a continuación:
a. Los Anexos del Contrato:
-
Anexo0_Glosario Tecnico
- Anexo1_Marco general y especificaciones técnicas
- Anexo2_Sedes y horarios
Anexo3_ModeloGestion UiPath
-
- Anexo4 Informes
Anexo5 TransicionServicio
-
- Anexo6 LíneaBaseRPA's
-
Anexo7_VolumetríaRPAsAnexo
- Anexo8 ANS ICD
- Anexo9 ModeloGobiernoDevOps
- Anexo10_ModeloGobiernoDataLake
- Anexo11_DocumentoEjecucionOE
- Anexo12_Manejo de Base de datos
Anexo13_Compromiso de Confidencialidad
-
- Anexo14 Formato Adenda
- Anexo15 Minuta Contrato
ISA INTERCOLOMBIA S.A. E.S.P. NIT: 900.667.590-1
Calle 12 Sur 18 - 168 Bloque 1 Pisos 2 y 3 Medellín, Colombia Tel: + 57 4 3252400
Línea única de atención: 018000 942001 Línea ética: opción 3 lineaetica@intercolombia.com
www.intercolombia.com
Docusign Envelope ID: A0E80020-8FB0-45B0-978B-B52BDC1B4E2C
isa INTERCOLOMBIA
b. El texto del Documento de Solicitud de Ofertas WS4914786530 y sus anexos, con los Adendos de 1 al 3.
c. La oferta de DATA KNOW S.A.S, incluidos sus anexos.
d. La comunicación de LA EMPRESA al CONTRATISTA, para la iniciación de las actividades objeto del Contrato.
e. La(s) Garantía(s) y Seguro(s) que se exigen en este Contrato, debidamente aceptadas por LA EMPRESA.
f. El cronograma de actividades.
g. Los Acuerdos de Niveles de Servicios - ANS -.
h. Las actas, demás acuerdos, correspondencia y los documentos que se produzcan y suscriban por las partes durante la ejecución de este Contrato. Todos los documentos mencionados obligan jurídicamente y son parte esencial e integral del Contrato. En casos de conflictos o discrepancias entre las disposiciones contenidas el Contrato, incluidos sus Anexos y las disposiciones contenidas en los demás documentos señalados, prevalecerán las disposiciones del Contrato, sus Anexos y Cláusulas Adicionales. De manera similar, en caso de conflictos o discrepancias entre las disposiciones del Contrato y las disposiciones contenidas en las resoluciones, decretos y normas legales aplicables al presente Contrato, prevalecerán las normas de mayor jerarquía.
32. RETENCIONES
En ejercicio del principio de la autonomía de la voluntad las partes acuerdan que LA EMPRESA podrá retener todo o parte de cualquier pago que deba realizar en favor de El Contratista en virtud del presente contrato, por efecto de la mora o del cumplimiento imperfecto, defectuoso o tardío o como garantía del cumplimiento de las obligaciones principales y accesorias contraídas por El Contratista. La retención procederá en los siguientes eventos:
a. Retardo o cumplimiento imperfecto, parcial, defectuoso o tardío de la obligación principal
b. Suspensión de las actividades objeto del Contrato sin causa justificada o por causa imputable al Contratista.
c. Inobservancia de las obligaciones laborales y de seguridad social adquiridas con el personal empleado en la ejecución del presente Contrato
d. Utilización o entrega de productos, materiales o equipos, o ejecución de procedimientos que no cumplan con las características o especificaciones técnicas ofertadas y aceptadas por LA EMPRESA.
e. No contar con el personal exigido para la ejecución del contrato, o presentándose una falta, no remplazarla dentro de los 30 días hábiles siguientes a su ocurrencia
f. Inobservancia del plazo de actividades definido para la entrega total o parcial de las
obras, bienes o servicios objeto de cada orden de servicio.
g. Inobservancia del Manual HSE para Contratistas de ISA o de los requisitos de HSE exigidos.
h. Desconocer los Acuerdos de Nivel de Servicio (ANS) u otros Acuerdos Contractuales.
PARÁGRAFO PRIMERO: La retención procederá por evento o por lo establecido en el "Anexo8_ANS ICD", por un (0,2%) diario o por ANS/ICD por debajo del mínimo aceptado,
ISA INTERCOLOMBIA S.A. E.S.P. NIT: 900.667.590-1
Calle 12 Sur 18 - 168 Bloque 1 Pisos 2 y 3 Medellín, Colombia Tel: + 57 4 3252400
Línea única de atención: 018000 942001 Línea ética: opción 3
lineaetica@intercolombia.com
www.intercolombia.com
Docusign Envelope ID: A0E80020-8FB0-45B0-978B-B52BDC1B4E2C
isa INTERCOLOMBIA
sobre el valor de cada orden de entrega. El Contratista deberá ponerse en situación de cumplimiento, so pena de la aplicación de la condición resolutoria expresa acordada en el contrato.
PARÁGRAFO SEGUNDO: La retención se hará efectiva sobre la factura más próxima a la ocurrencia del hecho que motivó la retención y que LA EMPRESA deba pagarle al CONTRATISTA con ocasión de la ejecución de presente contrato, luego de hacer las deducciones legales a que haya lugar. Cuando no existan saldos a favor del Contratista, LA EMPRESA podrá cobrar por la vía ejecutiva las sumas objeto de la retención para lo cual se considerará que el presente documento constituye título ejecutivo.
PARÁGRAFO TERCERO: Cuando hayan sido superadas las causas que originaron la retención y El Contratista se encuentre al día en sus obligaciones, LA EMPRESA efectuará la devolución de los valores retenidos.
PARÁGRAFO CUARTO: Por el hecho de hacerse efectiva la retención no se exonera El Contratista del cumplimiento de las obligaciones principales y accesorias derivadas del presente contrato ni de la indemnización de los perjuicios que LA EMPRESA haya sufrido, ni de la calificación negativa de su desempeño como CONTRATISTA.
33. PERFECCIONAMIENTO Y REQUISITOS DE EJECUCIÓN.
El presente Contrato requiere para su perfeccionamiento la firma de las partes.
ISA INTERCOLOMBIA S.A. E.S.P
DocuSigned by: DR. C
2BD56BC8FB8D472 DIANA ISABEL PIZARRO DIRECTORA SERVICIOS APROVISIONAMIENTO
DATA KNOW S.A.S
Firmado por: Carlos a Murillo 6 F8E9ECCC5949436 CARLOS ANDRES MURILLO GALLEGO REPRESENTANTE LEGAL
ISA INTERCOLOMBIA S.A. E.S.P. NIT: 900.667.590-1
Calle 12 Sur 18 - 168 Bloque 1 Pisos 2 y 3 Medellín, Colombia Tel: + 57 4 3252400
Línea única de atención: 018000 942001 Línea ética: opción 3 lineaetica@intercolombia.com
www.intercolombia.com
Docusign Envelope ID: A0E80020-8FB0-45B0-978B-B52BDC1B4E2C
isa INTERCOLOMBIA
Anexo 0 - Glosario Técnico
Contenido
1. Acuerdos de Niveles de Servicio 2
2. Las Empresas 3
3. Filial 3
4. Proveedor Analítica 3
5. Proveedor soporte infraestructura 3
6. Ambiente 3
7. Caso de Negocio: 4
8. Eventos relevantes 4
9. Jornada laboral 4
10. Jornada no laboral 4
11. Niveles de criticidad 5
12. Centro de Atención a Usuarios 5
13. Grupo Resolutor 5
14. Incidente 5
15. Solicitud / Requerimiento 5
16. Solicitudes o incidentes reabiertos
6
17. Problema. 6
18. Soporte de primer nivel: 6
19. Soporte de segundo nivel: 6
20. Soporte tercer nivel 6
21. Línea Base 7
22. Transición:
7
23. Estabilización del servicio: 7
24. Transformación: 7
25. Ventanas de mantenimiento 7
26. Soporte y Mantenimiento 8
27. Capacidad extendida 8
28. Trabajos especiales 8
29. IMACD 9
30. Desarrollo Tradicional 9
31. Desarrollo Ágil 9"""

In [71]:
response = client.chat.completions.create(
    model='gpt-4o',
     messages=[
        {
            "role": "system",
            "content": """Eres un agente especializado en extracción de campos de contratos, debes extraer siempre los siguientes campos en un json, RETORNA SOLO JSON siempre en un formato válido sin añadir o modificar los nombres de los campos: 
                [{
                        "Contrato/Orden": "ORD-2025-001",
                        "ContratoMarco": "CM-2024-045",
                        "NitProveedor": "900123456-1",
                        "NombreProveedor": "Soluciones TecnoPro S.A.S.",
                        "Objeto": "Suministro e instalación de equipos de cómputo",
                        "GestionGarantiasDoc": true,
                        "Cobertura": "Garantía extendida",
                        "DescripcionCobertura": "Cobertura por defectos de fabricación y mano de obra",
                        "CoberturaPara": "Hardware y accesorios",
                        "PorcentajeCobertura": 80,
                        "TextoTiempoAdicionalCobertura": "Meses adicionales de cobertura",
                        "TiempoAdicionalCobertura": 6,
                        "DescripcionValorDoc": "Valor total del contrato incluyendo IVA",
                        "ValorDoc": 150000000,
                        "Moneda": "COP",
                        "Plazo/VigenciaDoc": "12 meses",
                        "PlazoDoc": 12,
                        "FechaInicioCobertura": "2025-01-01",
                        "FechaFinCobertura": "2025-12-31",
                        "OrdenInicio": 1
                        }]

                        debe ser un Objeto por cada cobertura, 
                        Descripción de campos para la extracción de datos de documentos contractuales:
                        
                        Contrato/Orden: Número de 10 dígitos que se encuentra en el título del documento. En adjudicaciones, aparece después de "El número del Contrato es" y en órdenes de entrega, luego de "ORDEN DE ENTREGA No".

                        Contrato Marco: Campo opcional. En algunos casos, se puede encontrar después de "No CONTRATO" en órdenes de entrega.

                        NIT Proveedor: Se debe extraer de la lista contratopolizagarantia usando el número de Contrato/Orden como llave. Si no se encuentra, se deja vacío.

                        Nombre Proveedor: Igual que el NIT, se extrae desde la lista contratopolizagarantia, usando como clave el Contrato/Orden.

                        Objeto: También se obtiene desde contratopolizagarantia, a partir del Contrato/Orden.

                        Gestión Garantías Doc: Si en el documento se encuentra el título “GARANTÍAS, FIANZAS Y SEGUROS” y hay contenido debajo, se marca como “SI”; de lo contrario, se coloca “NO”.

                        Cobertura: Se refiere a cada subtítulo listado después del título “GARANTÍAS, FIANZAS Y SEGUROS” o “GARANTÍAS Y SEGUROS”. Si es un contrato marco, se deben crear múltiples filas para cada cobertura listada.

                        Descripción Cobertura: Texto que detalla la cobertura, incluyendo fecha, porcentaje y beneficiario. En documentos marco, también se extrae desde la lista correspondiente.

                        Cobertura Para: Indica si la cobertura aplica para el Contrato o para las Órdenes. Se debe traducir y dejar claro.

                        Porcentaje Cobertura: Se extrae desde la Descripción Cobertura y se convierte a valor numérico.

                        Texto Tiempo Adicional Cobertura: Fragmento textual donde se especifique plazo adicional de cobertura. Si no se encuentra, se deja vacío.

                        Tiempo Adicional Cobertura: Se traduce y extrae como está en el documento (ej. "6 meses adicionales").

                        Descripción Valor Doc: Texto ubicado después de frases como “El valor del contrato es” o “VALOR ANTES DE IMPUESTOS”.

                        Valor Doc: Se extrae del documento. Si el valor está en letras, se debe traducir a número. Si no es posible determinarlo, se marca como “INDETERMINADO”.

                        Moneda: Se obtiene a partir del texto asociado a la moneda (COP, USD, EUR), y puede estar antes o después del valor.

                        Plazo/Vigencia Doc: Texto encontrado después de expresiones como “VIGENCIA Y PLAZO DEL CONTRATO” o “PLAZO DE LA ORDEN DE ENTREGA”. Especifica duración y vigencia del contrato u orden.

                        Plazo Doc: De este campo se extrae el plazo en días, meses o años, y la fecha que se menciona para la constitución de garantías. Si no se encuentra, se deja vacío.

                        Fecha Inicio Cobertura: En contratos, se refiere a la fecha ubicada en secciones como “Resumen de eventos / Completado”. Para órdenes, también puede aparecer explícitamente.

                        Fecha Fin Cobertura: Si la cobertura es para el contrato, se calcula sumando el Plazo Doc a la Fecha Inicio Cobertura. Si no aplica, se deja vacío.

                        Orden Inicio: Se marca como “SI” si en el documento se menciona que hay una orden de inicio (por ejemplo: “ORDEN DE ENTREGA / ORDEN DE SERVICIOS”); en caso contrario, “NO”.



            """,
        },
        {
            "role": "user",
            "content": "Extrae los campos de este contenido fuente: " + content,
        }
    ]
)

In [72]:
response.choices[0].message.content

'```json\n[\n    {\n        "Contrato/Orden": "",\n        "ContratoMarco": "CS-112",\n        "NitProveedor": "901212206-9",\n        "NombreProveedor": "Data Know S.A.S.",\n        "Objeto": "Prestación de servicios de soporte, gestión, administración, monitoreo, mantenimiento y desarrollo de RPAs y analítica avanzada para ISA y sus empresas.",\n        "GestionGarantiasDoc": true,\n        "Cobertura": "Garantía de Cumplimiento",\n        "DescripcionCobertura": "Mediante esta garantía se amparan las obligaciones del contratista por la celebración de las órdenes de entrega. Vigente hasta la fecha de finalización del plazo de la orden más un mes.",\n        "CoberturaPara": "Contratos y órdenes de entrega",\n        "PorcentajeCobertura": 10,\n        "TextoTiempoAdicionalCobertura": "Un mes adicional desde la finalización del plazo.",\n        "TiempoAdicionalCobertura": 1,\n        "DescripcionValorDoc": "Valor del contrato es indeterminado; se calcula con base en la sumatoria de l

In [77]:
responseoai = response.choices[0].message.content.replace("json\n","")

In [82]:
responseoai

'```[\n    {\n        "Contrato/Orden": "",\n        "ContratoMarco": "CS-112",\n        "NitProveedor": "901212206-9",\n        "NombreProveedor": "Data Know S.A.S.",\n        "Objeto": "Prestación de servicios de soporte, gestión, administración, monitoreo, mantenimiento y desarrollo de RPAs y analítica avanzada para ISA y sus empresas.",\n        "GestionGarantiasDoc": true,\n        "Cobertura": "Garantía de Cumplimiento",\n        "DescripcionCobertura": "Mediante esta garantía se amparan las obligaciones del contratista por la celebración de las órdenes de entrega. Vigente hasta la fecha de finalización del plazo de la orden más un mes.",\n        "CoberturaPara": "Contratos y órdenes de entrega",\n        "PorcentajeCobertura": 10,\n        "TextoTiempoAdicionalCobertura": "Un mes adicional desde la finalización del plazo.",\n        "TiempoAdicionalCobertura": 1,\n        "DescripcionValorDoc": "Valor del contrato es indeterminado; se calcula con base en la sumatoria de los val

In [83]:
match = re.search(r'(\[.*\])', responseoai, re.S)
if not match:
    raise ValueError("No encontré un bloque JSON válido")
json_str = match.group(1)



In [79]:
import json

In [87]:
result = json.loads(json_str)

In [86]:
json.loads("[]")

[]

In [88]:
finaljson = json.dumps(result)

In [89]:
finaljson

'[{"Contrato/Orden": "", "ContratoMarco": "CS-112", "NitProveedor": "901212206-9", "NombreProveedor": "Data Know S.A.S.", "Objeto": "Prestaci\\u00f3n de servicios de soporte, gesti\\u00f3n, administraci\\u00f3n, monitoreo, mantenimiento y desarrollo de RPAs y anal\\u00edtica avanzada para ISA y sus empresas.", "GestionGarantiasDoc": true, "Cobertura": "Garant\\u00eda de Cumplimiento", "DescripcionCobertura": "Mediante esta garant\\u00eda se amparan las obligaciones del contratista por la celebraci\\u00f3n de las \\u00f3rdenes de entrega. Vigente hasta la fecha de finalizaci\\u00f3n del plazo de la orden m\\u00e1s un mes.", "CoberturaPara": "Contratos y \\u00f3rdenes de entrega", "PorcentajeCobertura": 10, "TextoTiempoAdicionalCobertura": "Un mes adicional desde la finalizaci\\u00f3n del plazo.", "TiempoAdicionalCobertura": 1, "DescripcionValorDoc": "Valor del contrato es indeterminado; se calcula con base en la sumatoria de los valores de las \\u00f3rdenes de entrega.", "ValorDoc": "IN